In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#-----
from DOVSAudit import DOVSAudit
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

In [ ]:
# outg_rec_nb = '13551139'
# outg_rec_nb = '13579980'
# outg_rec_nb = '13464549'
# outg_rec_nb = '13464904'
outg_rec_nb = '13759308'
# outg_rec_nb = '13749860'

outg_rec_nb = '13766571'

In [ ]:
# audit_i = DOVSAudit(
#     outg_rec_nb=outg_rec_nb
# )
# #-------------------------
# audit_i.load_dovs(
#     dovs_df           = None, 
#     dovs_df_info_dict = None
# )
# #-------------------------
# audit_i.load_ami_from_csvs(
#     paths                          = outg_rec_nb_to_files_dict[outg_rec_nb], 
#     slicers                        = None, 
#     ami_df_info_dict               = None, 
#     run_std_init                   = True, 
#     cols_and_types_to_convert_dict = None, 
#     to_numeric_errors              = 'coerce', 
#     drop_na_rows_when_exception    = True, 
#     drop_unnamed0_col              = True, 
#     pd_read_csv_kwargs             = None, 
#     make_all_columns_lowercase     = False, 
#     assert_all_cols_equal          = True, 
#     min_fsize_MB                   = None
# )
# #-------------------------
# if outg_rec_nb in outg_rec_nb_to_files_ede_dict.keys():
#     audit_i.load_ede_from_csvs(
#         paths                          = outg_rec_nb_to_files_ede_dict[outg_rec_nb], 
#         slicers                        = None, 
#         ede_df_info_dict               = None, 
#         run_std_init                   = True, 
#         cols_and_types_to_convert_dict = None, 
#         to_numeric_errors              = 'coerce', 
#         drop_na_rows_when_exception    = True, 
#         drop_unnamed0_col              = True, 
#         pd_read_csv_kwargs             = None, 
#         make_all_columns_lowercase     = False, 
#         assert_all_cols_equal          = True, 
#         min_fsize_MB                   = None
#     )
# #-------------------------

In [ ]:
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
audit_i = DOVSAudit(
    outg_rec_nb=outg_rec_nb
)
#-------------------------
audit_i.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
audit_i.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
audit_i.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)

In [ ]:
# audit_i.calculate_by_PN=False

In [ ]:
audit_i.build_best_ests_df()
audit_i.best_ests_df

# TMP DEV 0

In [ ]:
audit_i.best_ests_df[audit_i.best_ests_df['SN'].isin(
    audit_i.ami_df_i[audit_i.ami_df_i['trsf_pole_nb']=='38810897C00120']['serialnumber'].unique()
)]

In [ ]:
audit_i.ami_df_i[audit_i.ami_df_i['trsf_pole_nb']=='38810897C00120']['serialnumber'].unique()

In [ ]:
natsorted(audit_i.ami_df_i['trsf_pole_nb'].unique().tolist())

In [ ]:
ami_nonvee = AMINonVee(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args = None, 
    build_sql_function=AMINonVee_SQL.build_sql_usg, 
    build_sql_function_kwargs=dict(
        premise_nbs = ['020168331'],
        date_range = ['2024-01-25', '2024-01-29'], 
        aep_derived_uoms_and_idntfrs = [['KWH', 'DELIVERED']]
    ), 
    init_df_in_constructor=True, 
    save_args=False
)

fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig = fig, 
    ax  = ax, 
    data = ami_nonvee.df, 
    x = 'starttimeperiod', 
    y = 'value', 
    hue='serialnumber'
)

In [ ]:
ami_nonvee = AMINonVee(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args = None, 
    build_sql_function=AMINonVee_SQL.build_sql_usg, 
    build_sql_function_kwargs=dict(
        premise_nbs = ['020168331'],
        date_range = ['2024-01-25', '2024-01-29'], 
        aep_derived_uoms_and_idntfrs = [['KWH', 'RECEIVED']]
    ), 
    init_df_in_constructor=True, 
    save_args=False
)

fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig = fig, 
    ax  = ax, 
    data = ami_nonvee.df, 
    x = 'starttimeperiod', 
    y = 'value', 
    hue='serialnumber'
)

In [ ]:
ami_nonvee = AMINonVee(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args = None, 
    build_sql_function=AMINonVee_SQL.build_sql_usg, 
    build_sql_function_kwargs=dict(
        premise_nbs = ['020168331'],
        date_range = ['2024-01-28', '2024-01-29'], 
        aep_derived_uoms_and_idntfrs = ['VOLT']
    ), 
    init_df_in_constructor=True, 
    save_args=False
)

fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig = fig, 
    ax  = ax, 
    data = ami_nonvee.df, 
    x = 'starttimeperiod', 
    y = 'value', 
    hue='serialnumber'
)

# TMP DEV

In [ ]:
audit_i.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)

In [ ]:
audit_i.best_ests_df

In [ ]:
best_ests_df_w_mult = DOVSAudit.combine_PNs_in_best_ests_df(
    best_ests_df    = audit_i.best_ests_df, 
    likeness_thresh = audit_i.combine_by_PN_likeness_thresh, 
    SN_col          = 'SN', 
    PN_col          = 'PN', 
    i_outg_col      = 'i_outg'     
)

In [ ]:
best_ests_df_w_mult

In [ ]:
SN_col          = 'SN'
PN_col          = 'PN'
i_outg_col      = 'i_outg'   

In [ ]:
assert(
    (best_ests_df_w_mult
     .groupby([PN_col, i_outg_col])[SN_col].apply(list)
     .groupby([PN_col]).apply(list)
     .apply(lambda x: Utilities.are_all_lists_eq(x))
    ).all()
)

In [ ]:
(best_ests_df_w_mult
 .groupby([PN_col, i_outg_col])[SN_col].apply(list)
 .groupby([PN_col]).apply(list)
 .apply(lambda x: Utilities.are_all_lists_eq(x))
)

In [ ]:
def get_energy_provided_df(
    idfr_val, 
    idfr_field       = 'premise_nb', 
    date_pd          = None, 
    value_min        = 0.003, 
    cols_of_interest = None
):
    r"""
    Returns a pd.DataFrame object containing the KWH (=aep_derived_uom) RECEIVED (=aep_srvc_qlty_idntfr) for the premise(s)/serial number(s)
      for the time period date_pd (if not date_pd input, the past 30 days are used)
      
    idfr_val/idfr_field:
        idfr_val:
            The premise(s) or serial number(s) for which to acquire data.
            This can be a string (single PN/SN) or a list (multiple PNs/SNs)
        idfr_field:
            Must be one of the following: 
                'premise_nb', 'premise_nbs', 'serial_number', 'serial_numbers'
            This is NOT the field name in usage_nonvee.reading_ivl_nonvee, but instead the kwarg to be input
              into AMI_SQL.build_sql_ami (so the name idfr_field is a bit misleading)
              
        date_pd:
            Date period.
            Should be a list/tuple of length 2 or None
            If None, the past 30 days are used
            
        value_min:
            Minimum value to return (it seems error might be +- 0.002, so the default value of 0.003 is to remove
              any false non-zeros)
    """
    #-------------------------
    # For the purposes of this function, restrict idfr_field to
    #   : serial_number(s) or premise_nb(s)
    assert(idfr_field in ['serial_number', 'serial_numbers', 'premise_nb', 'premise_nbs'])
    #-------------------------
    if date_pd is None:
        date_pd = [
            pd.Timestamp('today').date()-pd.Timedelta('30D'), 
            pd.Timestamp('today').date()
        ]
    assert(Utilities.is_object_one_of_types(date_pd, [list, tuple]))
    assert(len(date_pd)==2)    
    #-------------------------
    build_sql_function_kwargs=dict(
        cols_of_interest             = cols_of_interest, 
        aep_derived_uoms_and_idntfrs = [['KWH', 'RECEIVED']], 
        date_range                   = date_pd, 
        value_min                    = value_min, 
        join_mp_args                 = False
    )
    build_sql_function_kwargs[idfr_field] = idfr_val
    #-------------------------
    ami = AMINonVee(
        df_construct_type         = DFConstructType.kRunSqlQuery, 
        build_sql_function        = AMINonVee_SQL.build_sql_usg, 
        build_sql_function_kwargs = build_sql_function_kwargs, 
        init_df_in_constructor    = True
    )
    ami_df = ami.df.copy()
    return ami_df

In [ ]:
def get_agg_energy_provided_df(
    idfr_val, 
    idfr_field       = 'premise_nb', 
    date_pd          = None, 
    value_min        = 0.003, 
    agg_fcns         = ['sum'], 
    grp_by_SN_and_PN = False
):
    r"""
    Returns a pd.DataFrame object containing the aggregate KWH (=aep_derived_uom) RECEIVED (=aep_srvc_qlty_idntfr) for the premise(s)/serial number(s)
      for the time period date_pd (if not date_pd input, the past 30 days are used).
    Groups the data by PN/SN (depending on idfr_field) and applies the aggregate functions in agg_fcns (default = ['sum']).
    The counts (including null) is always included.
      
    idfr_val/idfr_field:
        idfr_val:
            The premise(s) or serial number(s) for which to acquire data.
            This can be a string (single PN/SN) or a list (multiple PNs/SNs)
        idfr_field:
            Must be one of the following: 
                'premise_nb', 'premise_nbs', 'serial_number', 'serial_numbers'
            This is NOT the field name in usage_nonvee.reading_ivl_nonvee, but instead the kwarg to be input
              into AMI_SQL.build_sql_ami (so the name idfr_field is a bit misleading)
              
        date_pd:
            Date period.
            Should be a list/tuple of length 2 or None
            If None, the past 30 days are used
            
        value_min:
            Minimum value to return (it seems error might be +- 0.002, so the default value of 0.003 is to remove
              any false non-zeros)
    """
    #-------------------------
    # For the purposes of this function, restrict idfr_field to
    #   : serial_number(s) or premise_nb(s)
    assert(idfr_field in ['serial_number', 'serial_numbers', 'premise_nb', 'premise_nbs'])
    #-------------------------
    # Check that agg_fcns are all acceptable
    assert(set(agg_fcns).difference(set(['sum', 'sq_sum', 'mean', 'std', 'count']))==set())    
    #-------------------------
    if date_pd is None:
        date_pd = [
            pd.Timestamp('today').date()-pd.Timedelta('30D'), 
            pd.Timestamp('today').date()
        ]
    assert(Utilities.is_object_one_of_types(date_pd, [list, tuple]))
    assert(len(date_pd)==2)    
    #-------------------------
    idfr_col = None
    if idfr_field in ['serial_number', 'serial_numbers']:
        idfr_col = 'serialnumber'
    elif idfr_field in ['premise_nb', 'premise_nbs']:
        idfr_col = 'aep_premise_nb'
    else:
        assert(0)
    #-----
    value_col = 'value'
    #-----
    build_sql_function_kwargs=dict(
        cols_of_interest             = [idfr_col], 
        aep_derived_uoms_and_idntfrs = [['KWH', 'RECEIVED']], 
        date_range                   = date_pd, 
        value_min                    = value_min, 
        join_mp_args                 = False, 
        agg_cols_and_types           = {value_col:agg_fcns}, 
        groupby_cols                 = [idfr_col]
    )
    build_sql_function_kwargs[idfr_field] = idfr_val
    #-----
    if grp_by_SN_and_PN:
        build_sql_function_kwargs['cols_of_interest'] = ['serialnumber', 'aep_premise_nb']
        build_sql_function_kwargs['groupby_cols']     = ['serialnumber', 'aep_premise_nb']
    #-------------------------
    ami = AMINonVee(
        df_construct_type         = DFConstructType.kRunSqlQuery, 
        build_sql_function        = AMINonVee_SQL.build_sql_usg, 
        build_sql_function_kwargs = build_sql_function_kwargs, 
        init_df_in_constructor    = True
    )
    ami_df = ami.df.copy()
    return ami_df

In [ ]:
def is_an_energy_provider(
    idfr_val, 
    idfr_field       = 'premise_nb', 
    date_pd          = None, 
    value_min        = 0.003, 
):
    r"""
    Only used for a single PN/SN.
    If multiple are desired, one can use AMINonVee.get_agg_energy_provided_df
    Returns whether or not premise(s)/serial number(s) have provided AEP energy over the date period date_pd.
    Only entries with value_min are considered non-zero values.
    After applying such a cut, if the aggregate of 'value' over date_pd is non-zero, True is returned for the 
      premise(s)/serial number(s)
    
    idfr_val/idfr_field:
        idfr_val:
            The premise(s) or serial number(s) for which to acquire data.
            This can be a string (single PN/SN) or a list (multiple PNs/SNs)
        idfr_field:
            Must be one of the following: 
                'premise_nb', 'premise_nbs', 'serial_number', 'serial_numbers'
            This is NOT the field name in usage_nonvee.reading_ivl_nonvee, but instead the kwarg to be input
              into AMI_SQL.build_sql_ami (so the name idfr_field is a bit misleading)
              
        date_pd:
            Date period.
            Should be a list/tuple of length 2 or None
            If None, the past 30 days are used
            
        value_min:
            Minimum value to return (it seems error might be +- 0.002, so the default value of 0.003 is to remove
              any false non-zeros)
    """
    #-------------------------
    assert(isinstance(idfr_val, str))
    #-------------------------
    # For the purposes of this function, restrict idfr_field to
    #   : serial_number(s) or premise_nb(s)
    assert(idfr_field in ['serial_number', 'serial_numbers', 'premise_nb', 'premise_nbs'])
    #-------------------------
    idfr_col = None
    if idfr_field in ['serial_number', 'serial_numbers']:
        idfr_col = 'serialnumber'
    elif idfr_field in ['premise_nb', 'premise_nbs']:
        idfr_col = 'aep_premise_nb'
    else:
        assert(0)
    #-----
    value_col = 'value'
    #-------------------------
    agg_df = get_agg_energy_provided_df(
        idfr_val         = idfr_val, 
        idfr_field       = idfr_field, 
        date_pd          = date_pd, 
        value_min        = value_min, 
        agg_fcns         = ['sum'], 
        grp_by_SN_and_PN = False
    )
    #-------------------------
    if agg_df.shape[0] > 0:
        sum_value_col = 'sum_'+value_col
        assert((agg_df[sum_value_col]>0).all())
        return True
    else:
        return False

In [ ]:
(best_ests_df_w_mult
 .groupby([PN_col, i_outg_col])[SN_col].apply(list)
 .groupby([PN_col]).apply(list)
 .apply(lambda x: Utilities.are_all_lists_eq(x))
)

In [ ]:
is_an_energy_provider(
    idfr_val         = '789820769', 
    idfr_field       = 'serial_number', 
    date_pd          = None, 
    value_min        = 0.003
)

In [ ]:
is_an_energy_provider(
    idfr_val         = '995678718', 
    idfr_field       = 'serial_number', 
    date_pd          = None, 
    value_min        = 0.003
)

In [ ]:
best_ests_df_w_mult

In [ ]:
best_ests_df    = audit_i.best_ests_df.copy()
likeness_thresh = audit_i.combine_by_PN_likeness_thresh
SN_col = 'SN'
PN_col = 'PN'
i_outg_col = 'i_outg'
open_beg_col = 'open_beg'
open_end_col = 'open_end'


#-------------------------
# Make sure SN_col, PN_col, and i_outg_col all in best_ests_df
req_cols = [SN_col, PN_col, i_outg_col, open_beg_col, open_end_col]
assert(len(set(req_cols).difference(best_ests_df.columns.tolist()))==0)

# Determine which cols are mins and which are maxs
# NOTE: The method used for finding minmax_cols together with the assertion below assures that 
#       best_ests_df contains only SN_col, PN_col, i_outg_col and min/max cols
minmax_cols = [x for x in best_ests_df.columns.tolist() if x not in req_cols]
min_cols = [x for x in best_ests_df.columns.tolist() if x.endswith('_min')]
max_cols = [x for x in best_ests_df.columns.tolist() if x.endswith('_max')]
assert(len(set(minmax_cols).symmetric_difference(set(min_cols+max_cols)))==0)
assert(len(min_cols)==len(max_cols))

# Maintain the original order of columns in best_ests_df in the returned df
return_col_order = [x for x in best_ests_df.columns.tolist() if x in min_cols+max_cols]
return_col_order.extend([open_beg_col, open_end_col])
#-------------------------
# All columns in minmax_cols must be datetime type
# Below ensures all min/max columns are datetime
for col_i in min_cols+max_cols:
    dtype_i = best_ests_df[col_i].dtype
    if not (dtype_i is datetime.datetime or is_datetime64_dtype(dtype_i)):
        best_ests_df = Utilities_df.convert_col_type_w_pd_to_datetime(
            df=best_ests_df, 
            column=col_i, 
            inplace=True
        )

#-------------------------
# Really, the reduction operations below only need to be performed on PNs with more than one SN
# To save resources and time, split apart those needing reduced and those not
SNs_per_PN = best_ests_df[[SN_col, PN_col]].drop_duplicates()[PN_col].value_counts()
PNs_w_mult_SNs = SNs_per_PN[SNs_per_PN>1].index.tolist()

# If there are no PNs with multiple SNs, return best_ests_df (without SN_col, as the purpose of this
#   is to eliminate SN_col by grouping by premise)
# if len(PNs_w_mult_SNs)==0:
#     return best_ests_df.drop(columns=[SN_col]).sort_values(by=[PN_col, i_outg_col], ignore_index=True)

best_ests_df_w_mult  = best_ests_df[best_ests_df[PN_col].isin(PNs_w_mult_SNs)]
best_ests_df_wo_mult = best_ests_df[~best_ests_df[PN_col].isin(PNs_w_mult_SNs)]
assert(best_ests_df_w_mult.shape[0]+best_ests_df_wo_mult.shape[0]==best_ests_df.shape[0])

In [ ]:
#-------------------------
#return best_ests_df_w_mult
# Ensure that, when multiple sub-outages exist for a premise (i.e., when i_outg=0, 1, ...), 
#   all sub-outages share the same set of meters.
# See function documentation above for explanation of operations
assert(
    (best_ests_df_w_mult
     .groupby([PN_col, i_outg_col])[SN_col].apply(list)
     .groupby([PN_col]).apply(list)
     .apply(lambda x: Utilities.are_all_lists_eq(x))
    ).all()
)

In [ ]:
best_ests_df_w_mult

In [ ]:
best_ests_df_w_mult.groupby([PN_col, i_outg_col])[SN_col].apply(list)

In [ ]:
best_ests_df_w_mult.groupby([PN_col, i_outg_col])[SN_col].apply(list).groupby([PN_col]).apply(list)

In [ ]:
# If doesn't pass, then at least one meter (i.e., SN) did not lose power with the others
# This could happen if SN has its own power storage ability (e.g., solar panels with battery)
# In such a case, ignore the SN(s) EXCEPT if they have outage periods that do not overlap with
#   those found by the others (i.e., except when they have something new/interesting to offer)

In [ ]:
    (best_ests_df_w_mult
     .groupby([PN_col, i_outg_col])[SN_col].apply(list)
     .groupby([PN_col]).apply(list)
     .apply(lambda x: Utilities.are_all_lists_eq(x))
    )

In [ ]:
if not(
    (best_ests_df_w_mult
     .groupby([PN_col, i_outg_col])[SN_col].apply(list)
     .groupby([PN_col]).apply(list)
     .apply(lambda x: Utilities.are_all_lists_eq(x))
    ).all()
):
    print('Why the face?')

In [ ]:
best_ests_df_w_mult[min_cols].min().min()

In [ ]:
if pd.notna(best_ests_df_w_mult[min_cols].min().min()):
    date_pd = [
        best_ests_df_w_mult[min_cols].min().min()-pd.Timedelta('30D'), 
        best_ests_df_w_mult[min_cols].min().min()
    ]
else:
    date_pd = None

In [ ]:
agg_df = get_agg_energy_provided_df(
    idfr_val         = best_ests_df_w_mult[PN_col].unique().tolist(), 
    idfr_field       = 'premise_nbs', 
    date_pd          = date_pd, 
    value_min        = 0.003, 
    agg_fcns         = ['sum'], 
    grp_by_SN_and_PN = True
)

In [ ]:
agg_df_2 = get_agg_energy_provided_df(
    idfr_val         = best_ests_df_w_mult[SN_col].unique().tolist(), 
    idfr_field       = 'serial_numbers', 
    date_pd          = date_pd, 
    value_min        = 0.003, 
    agg_fcns         = ['sum'], 
    grp_by_SN_and_PN = True
)

In [ ]:
agg_df

In [ ]:
agg_df_2

In [ ]:
agg_df

In [ ]:
best_ests_df_w_mult

In [ ]:
agg_df

In [ ]:
min_sum_value = 0
provider_SNs = agg_df[agg_df['sum_value']>min_sum_value]['serialnumber'].unique().tolist()
provider_SNs

In [ ]:
best_ests_df_w_mult[best_ests_df_w_mult[SN_col].isin(provider_SNs)]

In [ ]:
df1 = get_energy_provided_df(
#     idfr_val         = '020168331', 
#     idfr_field       = 'premise_nb', 
#     idfr_val         = '995678718', 
#     idfr_val         = '789820769', 
    idfr_val         = ['995678718', '789820769'], 
    idfr_field       = 'serial_number', 
    date_pd          = None, 
    value_min        = 0.003, 
    cols_of_interest = None
)

In [ ]:
df2 = get_agg_energy_provided_df(
#     idfr_val         = '020168331', 
#     idfr_field       = 'premise_nb', 
#     idfr_val         = '995678718', 
    idfr_val         = '789820769', 
#     idfr_val         = ['995678718', '789820769'], 
    idfr_field       = 'serial_number', 
    date_pd          = None, 
    value_min        = 0.003, 
    agg_fcns         = ['sum']
)

In [ ]:
df1

In [ ]:
(df2['sum_value']>0).all()

In [ ]:
df2.loc[df2['sum_value']>0, 'sum_value']

In [ ]:
idfr_val = '020168331'
idfr_field = 'premise_nb' #NOT REALLY A FIELD, BUT A KWARG FOR build_sql_ami
date_pd = None
value_min = 0.003

In [ ]:
if date_pd is None:
    date_pd = [
        pd.Timestamp('today').date()-pd.Timedelta('30D'), 
        pd.Timestamp('today').date()
    ]
assert(Utilities.is_object_one_of_types(date_pd, [list, tuple]))
assert(len(date_pd)==2)

In [ ]:
# For the purposes of this function, restrict idfr_field to
#   : serial_number(s) or premise_nb(s)
assert(idfr_field in ['serial_number', 'serial_numbers', 'premise_nb', 'premise_nbs'])
build_sql_function_kwargs=dict(
    cols_of_interest             = None, 
    aep_derived_uoms_and_idntfrs = [['KWH', 'RECEIVED']], 
    date_range                   = date_pd, 
    value_min                    = value_min, 
    join_mp_args                 = False
)
build_sql_function_kwargs[idfr_field] = idfr_val

In [ ]:
build_sql_function_kwargs

In [ ]:
# For the purposes of this function, restrict idfr_field to
#   : serial_number(s) or premise_nb(s)
assert(idfr_field in ['serial_number', 'serial_numbers', 'premise_nb', 'premise_nbs'])
build_sql_function_kwargs=dict(
    cols_of_interest             = ['aep_premise_nb'], 
    aep_derived_uoms_and_idntfrs = [['KWH', 'RECEIVED']], 
    date_range                   = date_pd, 
    value_min                    = value_min, 
    join_mp_args                 = False, 
    agg_cols_and_types           = {'value':['sum']}, 
    groupby_cols                 = ['aep_premise_nb']
)
build_sql_function_kwargs[idfr_field] = idfr_val

In [ ]:
build_sql_function_kwargs

In [ ]:
ami = AMINonVee(
    df_construct_type         = DFConstructType.kRunSqlQuery, 
    build_sql_function        = AMINonVee_SQL.build_sql_usg, 
    build_sql_function_kwargs = build_sql_function_kwargs, 
    init_df_in_constructor    = True
)

In [ ]:
print(ami.get_sql_statement())

In [ ]:
ami.df

# TMP DEV END

In [ ]:
audit_i.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)

In [ ]:
audit_i.best_ests_df_w_keep_info

In [ ]:
audit_i.best_ests_df

In [ ]:
audit_i.resolve_overlapping_audits()

In [ ]:
audit_i.best_ests_df.shape

In [ ]:
audit_i.best_ests_df_w_keep_info.shape

In [ ]:
audit_i.best_ests_df

In [ ]:
means_df, best_ests_df_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
    best_ests_df                  = audit_i.best_ests_df, 
    eps_min                       = 5, 
    min_samples                   = 2, 
    ests_to_include_in_clustering = ['winner_min', 'winner_max'],
    ests_to_include_in_output     = [
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df  = True
)

In [ ]:
means_df

In [ ]:
calculate_by_PN=True
detailed_summary_df_i = DOVSAudit.build_detailed_summary_df(
    means_df              = means_df, 
    best_ests_df_w_db_lbl = best_ests_df_w_db_lbl,
    CI_tot                = audit_i.ci, 
    CMI_tot               = audit_i.cmi, 
    n_PNs_ami             = audit_i.ami_df_i['aep_premise_nb'].nunique(),
    outg_rec_nb           = audit_i.outg_rec_nb, 
    dovs_df_i             = audit_i.dovs_df_i, 
    warnings_flag         = audit_i.warnings_flag, 
    db_label_col          = 'db_label', 
    winner_min_col        = 'winner_min', 
    winner_max_col        = 'winner_max', 
    PN_col                = 'PN' if calculate_by_PN else 'SN', 
    i_outg_col            = 'i_outg'
)

In [ ]:
detailed_summary_df_i

In [ ]:
audit_i.outg_rec_nb

In [ ]:
audit_i.dovs_outg_t_beg_end

# IF ANY has keep==False, then further analysis is needed
Probably want to output/notify user

In [ ]:
(audit_i.best_ests_df_w_keep_info['keep']==False).sum()

In [ ]:
overlap_outg_rec_nbs = list(set(audit_i.best_ests_df_w_keep_info['overlap_DOVS'].sum()))
overlap_outg_rec_nbs

In [ ]:
audit_i.outg_rec_nb

# Option to either:
1. Build ovrlp_audit_a using ami_df_i from audit_i
    - less safe method, but checks can be put in place to switch to method 2 if criteria not met
    - should run quicker
    - currently, ovrlp_audit_a below

2. Build ovrlp_audit_a from scratch
    - safer method
    - almost certainly takes longer
    - - currently, ovrlp_audit_a2 below
    
For 13551139, both options give same result!

In [ ]:
# OPTION 1 (as described above)
#-------------------------
j=0
#-------------------------
ovrlp_audit_a = DOVSAudit(
    outg_rec_nb=overlap_outg_rec_nbs[j]
)
#-------------------------
ovrlp_audit_a.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
PNs_a = ovrlp_audit_a.PNs_dovs
#-----
ami_df_a = audit_i.ami_df_i.drop(columns=['removed_due_to_overlap']).copy()
ami_df_a = ami_df_a[ami_df_a[audit_i.ami_df_info_dict['PN_col']].isin(PNs_a)]
#-----
ede_df_a = audit_i.ede_df_i.copy()
ede_df_a = ede_df_a[ede_df_a[audit_i.ede_df_info_dict['PN_col']].isin(PNs_a)]
#-------------------------
ovrlp_audit_a.load_ami_from_df(
    ami_df_i         = ami_df_a, 
    ami_df_info_dict = audit_i.ami_df_info_dict, 
    run_std_init     = False, 
    slicers          = None
)
#-----
ovrlp_audit_a.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-----
ovrlp_audit_a.load_ede_from_df(
    ede_df_i         = ede_df_a, 
    ede_df_info_dict = audit_i.ede_df_info_dict, 
    run_std_init     = False
)
#-------------------------
ovrlp_audit_a.build_best_ests_df()
#-------------------------
ovrlp_audit_a.identify_overlaps(
    overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage
)

In [ ]:
ovrlp_audit_a.best_ests_df_w_keep_info

In [ ]:
# OPTION 2 (as described above)
#-------------------------
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
j=0
#-------------------------
ovrlp_audit_a2 = DOVSAudit(
    outg_rec_nb=overlap_outg_rec_nbs[j]
)
#-------------------------
ovrlp_audit_a2.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
ovrlp_audit_a2.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
ovrlp_audit_a2.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)
#-------------------------
ovrlp_audit_a2.build_best_ests_df()
#-------------------------
ovrlp_audit_a2.identify_overlaps(
    overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage
)

In [ ]:
ovrlp_audit_a2.best_ests_df

In [ ]:
ovrlp_audit_a2.best_ests_df_w_keep_info

In [ ]:
print(ovrlp_audit_a.best_ests_df_w_keep_info.equals(ovrlp_audit_a2.best_ests_df_w_keep_info))
print(ovrlp_audit_a.best_ests_df.equals(ovrlp_audit_a2.best_ests_df))

In [ ]:
# Columns whose values can be used to uniquely identify an entry in best_ests_df_w_keep_info
unq_idfr_cols = ['PN', 'i_outg']

In [ ]:
(audit_i.best_ests_df_w_keep_info['keep']==False).sum()

In [ ]:
be_df_wki_i       = audit_i.best_ests_df_w_keep_info.copy()
ovrlp_be_df_wki_a = ovrlp_audit_a.best_ests_df_w_keep_info.copy()
# #-------------------------
# # Sanity check: Make sure audit_i.outg_rec_nb found somewhere in ovrlp_be_df_wki_a
# assert((ovrlp_be_df_wki_a['overlap_DOVS'].apply(lambda x: audit_i.outg_rec_nb in x)).any())
# #-----
# # Only interested in entires in ovrlp_be_df_wki_a which overlap with audit_i.outg_rec_nb
# ovrlp_be_df_wki_a=ovrlp_be_df_wki_a[
#     ovrlp_be_df_wki_a['overlap_DOVS'].apply(lambda x: audit_i.outg_rec_nb in x)
# ]
# #-------------------------
# # Sanity check: Make sure ovrlp_audit_a.outg_rec_nb found somewhere in be_df_wki_i
# assert((be_df_wki_i['overlap_DOVS'].apply(lambda x: ovrlp_audit_a.outg_rec_nb in x)).any())
# #-----
# # Only interested in entires in be_df_wki_i which overlap with ovrlp_audit_a.outg_rec_nb
# be_df_wki_i=be_df_wki_i[
#     be_df_wki_i['overlap_DOVS'].apply(lambda x: ovrlp_audit_a.outg_rec_nb in x)
# ]

# Resolve overlaps

In [ ]:
audit_a = copy.deepcopy(audit_i)
audit_b = copy.deepcopy(ovrlp_audit_a)
unq_idfr_cols = ['PN', 'i_outg']
PN_col='PN'
t_min_col='winner_min'
t_max_col='winner_max'
keep_col='keep'
overlap_DOVS_col='overlap_DOVS'
open_beg_col = 'open_beg'
open_end_col = 'open_end'

In [ ]:
audit_a1 = copy.deepcopy(audit_i)
audit_a2 = copy.deepcopy(audit_i)
audit_a3 = copy.deepcopy(audit_i)
audit_a4 = copy.deepcopy(audit_i)

audit_b1 = copy.deepcopy(ovrlp_audit_a)
audit_b2 = copy.deepcopy(ovrlp_audit_a)
audit_b3 = copy.deepcopy(ovrlp_audit_a)
audit_b4 = copy.deepcopy(ovrlp_audit_a)

In [ ]:
audit_a1.best_ests_df[audit_a1.best_ests_df['PN']=='960256588']

In [ ]:
audit_a1.best_ests_df_w_keep_info[audit_a1.best_ests_df_w_keep_info['PN']=='960256588']

In [ ]:
PNs_to_resolve_A = audit_a.best_ests_df_w_keep_info[audit_a.best_ests_df_w_keep_info['keep']==False]['PN'].unique().tolist()
# PNs_to_resolve_B = audit_a.best_ests_df_w_keep_info['PN'].unique().tolist()
PNs_to_resolve_B = None

In [ ]:
adjustment_dicts_1 = DOVSAudit.resolve_overlapping_audits(
    audit_a          = audit_a1, 
    audit_b          = audit_b1, 
    PNs_to_resolve   = PNs_to_resolve_A, 
    unq_idfr_cols    = unq_idfr_cols, 
    PN_col           = PN_col, 
    t_min_col        = t_min_col, 
    t_max_col        = t_max_col, 
    keep_col         = keep_col, 
    overlap_DOVS_col = overlap_DOVS_col, 
    open_beg_col     = open_beg_col, 
    open_end_col     = open_end_col
)

In [ ]:
adjustment_dicts_2 = DOVSAudit.resolve_overlapping_audits_v2(
    audit_a          = audit_a2, 
    audit_b          = audit_b2, 
    PNs_to_resolve   = PNs_to_resolve_A, 
    unq_idfr_cols    = unq_idfr_cols, 
    PN_col           = PN_col, 
    t_min_col        = t_min_col, 
    t_max_col        = t_max_col, 
    keep_col         = keep_col, 
    overlap_DOVS_col = overlap_DOVS_col, 
    open_beg_col     = open_beg_col, 
    open_end_col     = open_end_col
)

In [ ]:
adjustment_dicts_3 = DOVSAudit.resolve_overlapping_audits(
    audit_a          = audit_a3, 
    audit_b          = audit_b3, 
    PNs_to_resolve   = PNs_to_resolve_B, 
    unq_idfr_cols    = unq_idfr_cols, 
    PN_col           = PN_col, 
    t_min_col        = t_min_col, 
    t_max_col        = t_max_col, 
    keep_col         = keep_col, 
    overlap_DOVS_col = overlap_DOVS_col, 
    open_beg_col     = open_beg_col, 
    open_end_col     = open_end_col
)

In [ ]:
adjustment_dicts_4 = DOVSAudit.resolve_overlapping_audits_v2(
    audit_a          = audit_a4, 
    audit_b          = audit_b4, 
    PNs_to_resolve   = PNs_to_resolve_B, 
    unq_idfr_cols    = unq_idfr_cols, 
    PN_col           = PN_col, 
    t_min_col        = t_min_col, 
    t_max_col        = t_max_col, 
    keep_col         = keep_col, 
    overlap_DOVS_col = overlap_DOVS_col, 
    open_beg_col     = open_beg_col, 
    open_end_col     = open_end_col
)

In [ ]:
# Expect True
print(audit_a1.best_ests_df_w_keep_info.equals(audit_a2.best_ests_df_w_keep_info))
print(audit_a1.best_ests_df_w_keep_info.equals(audit_a3.best_ests_df_w_keep_info))
print(audit_a1.best_ests_df_w_keep_info.equals(audit_a4.best_ests_df_w_keep_info))

print(audit_b1.best_ests_df_w_keep_info.equals(audit_b2.best_ests_df_w_keep_info))
print(audit_b1.best_ests_df_w_keep_info.equals(audit_b3.best_ests_df_w_keep_info))
print(audit_b1.best_ests_df_w_keep_info.equals(audit_b4.best_ests_df_w_keep_info))

In [ ]:
audit_i.best_ests_df_w_keep_info[audit_i.best_ests_df_w_keep_info['keep']==True]

In [ ]:
audit_a1.best_ests_df_w_keep_info

In [ ]:
audit_a1.best_ests_df

In [ ]:
audit_a1.best_ests_df = audit_a1.best_ests_df_w_keep_info[audit_a1.best_ests_df_w_keep_info['keep']==True].drop(columns=['overlap_DOVS', 'overlap_times', 'keep'])

In [ ]:
audit_a1.best_ests_df_w_keep_info

In [ ]:
audit_a1.best_ests_df_w_keep_info[audit_a1.best_ests_df_w_keep_info['resolved']==True]

In [ ]:
audit_a1.best_ests_df_w_keep_info[audit_a1.best_ests_df_w_keep_info['resolved_details']!='']

In [ ]:
audit_a1.best_ests_df_w_keep_info[audit_a1.best_ests_df_w_keep_info['PN']=='960256588']

In [ ]:
adjustment_dicts_1['960256588']

In [ ]:
adjustment_dicts_1['960256588']['to_adjust_a']

In [ ]:
adjustment_dicts_1['960256588']['to_adjust_b']

In [ ]:
audit_a1.best_ests_df_w_keep_info['resolved_details'].value_counts()

In [ ]:
audit_a1.best_ests_df_w_keep_info[
    (audit_a1.best_ests_df_w_keep_info['keep']==True) |
    (audit_a1.best_ests_df_w_keep_info['resolved']==True)
]

In [ ]:
audit_a1.best_ests_df_w_keep_info[
    (audit_a1.best_ests_df_w_keep_info['keep']==True)
]

In [ ]:
FUCK

In [ ]:
# audit_a1.best_ests_df = audit_a1.best_ests_df_w_keep_info[
#     (audit_a1.best_ests_df_w_keep_info['keep']==True) |
#     (audit_a1.best_ests_df_w_keep_info['resolved']==True)
# ].drop(columns=['overlap_DOVS', 'overlap_times', 'keep'])

# audit_a1.best_ests_df = audit_a1.best_ests_df_w_keep_info[
#     (audit_a1.best_ests_df_w_keep_info['keep']==True)
# ].drop(columns=['overlap_DOVS', 'overlap_times', 'keep'])

audit_a1.best_ests_df = audit_a1.best_ests_df_w_keep_info[
    (audit_a1.best_ests_df_w_keep_info['keep']==True) |
    (audit_a1.best_ests_df_w_keep_info['resolved_details']=='Unresolved by current methods')
].drop(columns=['overlap_DOVS', 'overlap_times', 'keep'])

In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
best_ests_df = audit_i.best_ests_df.copy()
ami_df_i = audit_i.ami_df_i.copy()
#-----
dovs_outg_t_beg_end = audit_i.dovs_outg_t_beg_end
dovs_outg_t_beg = dovs_outg_t_beg_end[0]
dovs_outg_t_end = dovs_outg_t_beg_end[1]
#-----
outage_nb  = audit_i.outage_nb
n_PNs_dovs = audit_i.n_PNs_dovs
ci_dovs    = audit_i.ci_cmi_dovs[0]
cmi_dovs   = audit_i.ci_cmi_dovs[1]
ci_ami     = best_ests_df['PN'].nunique() 
cmi_ami    = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
threshold=95
#-------------------------
outg_SNs = best_ests_df['PN'].unique().tolist()
ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]
#-------------------------
means_df, best_ests_df_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=True
)
#-------------------------
n_PNs_w_power_srs = DOVSAudit.build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)

In [ ]:
fig_num=0
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = means_df, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

fig_num += 1

In [ ]:
best_ests_df = audit_b.best_ests_df_w_keep_info.copy()
ami_df_i = audit_b.ami_df_i.copy()
#-----
dovs_outg_t_beg_end = audit_b.dovs_outg_t_beg_end
dovs_outg_t_beg = dovs_outg_t_beg_end[0]
dovs_outg_t_end = dovs_outg_t_beg_end[1]
#-----
outage_nb  = audit_b.outage_nb
n_PNs_dovs = audit_b.n_PNs_dovs
ci_dovs    = audit_b.ci_cmi_dovs[0]
cmi_dovs   = audit_b.ci_cmi_dovs[1]
ci_ami     = best_ests_df['PN'].nunique() 
cmi_ami    = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
threshold=95
#-------------------------
outg_SNs = best_ests_df['PN'].unique().tolist()
ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]
#-------------------------
means_df, best_ests_df_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=True
)
#-------------------------
n_PNs_w_power_srs = DOVSAudit.build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)

In [ ]:
fig_num=0
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = means_df, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

fig_num += 1

In [ ]:
adjustment_dicts_1['960256588']

In [ ]:
adjustment_dicts_1['960256588']['to_adjust_a']

In [ ]:
adjustment_dicts_1['960256588']['to_adjust_b']

In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=0)
fig,ax = AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='aep_premise_nb'
)
ax.legend().set_visible(False)
#-----
ax.axvline(pd.to_datetime('2023-07-03 15:34:00'), color='lightblue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='lightblue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='blue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 19:07:00'), color='blue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 19:50:00'), color='green', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 21:11:00'), color='green', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 04:04:00'), color='red', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 07:00:00'), color='red', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 09:07:00'), color='orange', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 11:30:00'), color='orange', ls='--')

In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=0)
fig,ax = AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i[ami_df_i['aep_premise_nb']=='960376596'], 
    x='starttimeperiod_local', 
    y='value', 
    hue='aep_premise_nb'
)
ax.legend().set_visible(False)
#-----
ax.axvline(pd.to_datetime('2023-07-03 15:34:00'), color='lightblue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='lightblue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='blue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 19:07:00'), color='blue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 19:50:00'), color='green', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 21:11:00'), color='green', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 04:04:00'), color='red', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 07:00:00'), color='red', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 09:07:00'), color='orange', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 11:30:00'), color='orange', ls='--')

In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=0)
fig,ax = AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i[ami_df_i['aep_premise_nb']=='960256588'], 
    x='starttimeperiod_local', 
    y='value', 
    hue='aep_premise_nb'
)
ax.legend().set_visible(False)
#-----
ax.axvline(pd.to_datetime('2023-07-03 15:34:00'), color='lightblue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='lightblue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='blue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 19:07:00'), color='blue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 19:50:00'), color='green', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 21:11:00'), color='green', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 04:04:00'), color='red', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 07:00:00'), color='red', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 09:07:00'), color='orange', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 11:30:00'), color='orange', ls='--')

In [ ]:
audit_a = copy.deepcopy(audit_i)
audit_b = copy.deepcopy(ovrlp_audit_a)
PN_i = '960376596'

implement_resolutions=False
unq_idfr_cols = ['PN', 'i_outg']
PN_col='PN'
t_min_col='winner_min'
t_max_col='winner_max'
keep_col='keep'
overlap_DOVS_col='overlap_DOVS'
open_beg_col='open_beg'
open_end_col='open_end'

In [ ]:
# Make sure best_ests_df_w_keep_info exists for each
#  Below will fail if object is None or empty DF
assert(audit_a.best_ests_df_w_keep_info.shape[0]>0)
assert(audit_b.best_ests_df_w_keep_info.shape[0]>0)
#-------------------------
be_df_cols = unq_idfr_cols + list(set([t_min_col, t_max_col, open_beg_col, open_end_col]).difference(set(unq_idfr_cols)))
assert(set(be_df_cols).difference(set(audit_a.best_ests_df_w_keep_info.columns.tolist()))==set())
assert(set(be_df_cols).difference(set(audit_b.best_ests_df_w_keep_info.columns.tolist()))==set())
#-----
return_cols = be_df_cols + ['adjustment', 'resolved', 'resolved_details']
#----------------------------------------------------------------------------------------------------
outg_rec_nb_a         = audit_a.outg_rec_nb
dovs_outg_t_beg_end_a = audit_a.dovs_outg_t_beg_end
#-----
outg_rec_nb_b         = audit_b.outg_rec_nb
dovs_outg_t_beg_end_b = audit_b.dovs_outg_t_beg_end
#--------------------------------------------------        
# Grab the best_ests_df_w_keep_info object from each audit
be_df_a = audit_a.best_ests_df_w_keep_info[audit_a.best_ests_df_w_keep_info[PN_col]==PN_i].copy()
be_df_b = audit_b.best_ests_df_w_keep_info[audit_b.best_ests_df_w_keep_info[PN_col]==PN_i].copy()
#--------------------------------------------------
resolved_i = False
resolved_i_details = ''
#-----
to_adjust_a = DOVSAudit.set_initial_to_adjust_df(
    be_df       = be_df_a, 
    be_df_cols  = be_df_cols, 
    addtnl_cols = [keep_col]
)
to_adjust_b = DOVSAudit.set_initial_to_adjust_df(
    be_df       = be_df_b, 
    be_df_cols  = be_df_cols, 
    addtnl_cols = [keep_col]
)


In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=0)
fig,ax = AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i[ami_df_i['aep_premise_nb']=='960376596'], 
    x='starttimeperiod_local', 
    y='value', 
    hue='aep_premise_nb'
)
ax.legend().set_visible(False)
#-----
ax.axvline(pd.to_datetime('2023-07-03 15:34:00'), color='lightblue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='lightblue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='blue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 19:07:00'), color='blue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 19:50:00'), color='green', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 21:11:00'), color='green', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 04:04:00'), color='red', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 07:00:00'), color='red', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 09:07:00'), color='orange', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 11:30:00'), color='orange', ls='--')

In [ ]:
audit_1.

In [ ]:
audit_1.ci_cmi_dovs

In [ ]:
FUCK

In [ ]:
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
audit_1 = DOVSAudit(
#     outg_rec_nb='13464904'
    
#     outg_rec_nb='13582178'
    outg_rec_nb='13582332'
    
#     outg_rec_nb='13615597'
#     outg_rec_nb='13686253'
#     outg_rec_nb='13686882'
#     outg_rec_nb='13691684'
#     outg_rec_nb='13683659'
#     outg_rec_nb='13699234'
    
#     outg_rec_nb='13602414'
#     outg_rec_nb='13602659'
    
#     outg_rec_nb='13575999'
#     outg_rec_nb='13576413'
    
#     outg_rec_nb='13551139'
#     outg_rec_nb='13551140'
    
#     outg_rec_nb='13581052'
#     outg_rec_nb='13581346'
    
#     outg_rec_nb='13593009'
#     outg_rec_nb='13593297'
    
#     outg_rec_nb='13615597'
#     outg_rec_nb='13615810'
)
#-------------------------
audit_1.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
audit_1.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
audit_1.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)
#-------------------------
audit_1.build_best_ests_df()
#-------------------------


In [ ]:
audit_1.calculate_by_PN

In [ ]:
audit_1.best_ests_df

In [ ]:
audit_1.best_ests_df[audit_1.best_ests_df['PN']=='023368334']

In [ ]:
fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage_around_outage(
    fig=fig,
    ax=ax,
    data=audit_1.ami_df_i,
    x='starttimeperiod_local',
    y='value',
    hue='serialnumber',
    out_t_beg=audit_1.dovs_outg_t_beg_end[0],
    out_t_end=audit_1.dovs_outg_t_beg_end[1],
    expand_time=pd.Timedelta('1h'),
)
ax.legend().set_visible(False)

In [ ]:
fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage_around_outage(
    fig=fig,
    ax=ax,
    data=audit_1.ami_df_i,
    x='starttimeperiod_local',
    y='value',
    hue='serialnumber',
    out_t_beg=audit_1.dovs_outg_t_beg_end[0],
    out_t_end=audit_1.dovs_outg_t_beg_end[1],
    expand_time=pd.Timedelta('1h'),
)
ax.legend().set_visible(False)
#-----
ax = AMINonVee.draw_outage_limits_on_ax(
    ax                         = ax, 
    out_t_beg                  = audit_1.dovs_outg_t_beg_end[0], 
    out_t_end                  = audit_1.dovs_outg_t_beg_end[1], 
    plot_t_beg                 = None, 
    plot_t_end                 = None, 
    draw_outage_limits_kwargs  = dict(alpha=1.0, linewidth=5.0, ymin=0.95), 
    include_outage_limits_text = dict(
        out_t_beg_text = 'OUTG_REC_NB_i', 
        out_t_beg_ypos=(1.0-0.06, 'ax_coord'), 
        out_t_beg_va='top', 
        out_t_beg_ha='center', 
        out_t_beg_color='red', 
        #-----
        out_t_end_text = 'OUTG_REC_NB_i', 
        out_t_end_ypos=(1.0-0.06, 'ax_coord'), 
        out_t_end_va='top', 
        out_t_end_ha='center', 
        out_t_end_color='green', 
    ), 
    out_t_beg_line_color       = 'red', 
    out_t_end_line_color       = 'green',
    text_only                  = False
)

In [ ]:
audit_1.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)

In [ ]:
audit_1.resolve_overlapping_audits()

In [ ]:
audit_1.best_ests_df_w_keep_info

In [ ]:
audit_i=copy.deepcopy(audit_1)

In [ ]:
best_ests_df = audit_i.best_ests_df.copy()
ami_df_i = audit_i.ami_df_i.copy()
#-----
dovs_outg_t_beg_end = audit_i.dovs_outg_t_beg_end
dovs_outg_t_beg = dovs_outg_t_beg_end[0]
dovs_outg_t_end = dovs_outg_t_beg_end[1]
#-----
outage_nb  = audit_i.outage_nb
n_PNs_dovs = audit_i.n_PNs_dovs
ci_dovs    = audit_i.ci_cmi_dovs[0]
cmi_dovs   = audit_i.ci_cmi_dovs[1]
ci_ami     = best_ests_df['PN'].nunique() 
cmi_ami    = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
threshold=95
#-------------------------
outg_SNs = best_ests_df['PN'].unique().tolist()
ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]
#-------------------------
means_df, best_ests_df_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=True
)
#-------------------------
n_PNs_w_power_srs = DOVSAudit.build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)

In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
list(set(audit_i.best_ests_df_w_keep_info['overlap_DOVS'].sum()))

In [ ]:
ovrlp_dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=list(set(audit_i.best_ests_df_w_keep_info['overlap_DOVS'].sum())), 
        include_premise=True
    ), 
    build_consolidated=True
)
other_dovs_events_df = ovrlp_dovs.df.reset_index().copy()

In [ ]:
audit_i.best_ests_df_w_keep_info

In [ ]:
fig_num=0
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = means_df, 
    outg_rec_nb          = audit_i.outg_rec_nb, 
    outage_nb            = audit_i.outage_nb, 
    n_PNs_dovs           = audit_i.n_PNs_dovs, 
    ci_dovs              = audit_i.ci_cmi_dovs[0], 
    cmi_dovs             = audit_i.ci_cmi_dovs[0], 
    ci_ami               = audit_i.ci, 
    cmi_ami              = audit_i.cmi, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True), 
    other_dovs_events_df = other_dovs_events_df, 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

for ax_i in axs:
    ax_i.xaxis.set_tick_params(labelbottom=True)

fig_num += 1

In [ ]:
means_df

In [ ]:
best_ests_df_w_db_lbl

In [ ]:
ests_to_include_in_output=[
            'winner_min', 'winner_max', 
            'conservative_min', 'conservative_max', 
            'zero_times_min', 'zero_times_max'
        ]

In [ ]:
best_ests_df_w_db_lbl.groupby(['db_label'], as_index=True)[ests_to_include_in_output].mean()

In [ ]:
best_ests_df_w_db_lbl.groupby(['db_label'], as_index=True).size()

In [ ]:
best_ests_df_w_db_lbl.groupby(['db_label'], as_index=True).count()

In [ ]:
counts_srs = best_ests_df_w_db_lbl.groupby(['db_label'], as_index=True).size()

In [ ]:
counts_srs.name = 'counts'

In [ ]:
means_df = pd.merge(
    means_df, 
    counts_srs, 
    left_index=True, 
    right_index=True, 
    how='left'
)

In [ ]:
n_PNs_w_power_srs

In [ ]:
audit_1.best_ests_df_w_keep_info

In [ ]:
# def draw_outage_limits_on_ax(
#     ax, 
#     out_t_beg, 
#     out_t_end, 
#     plot_t_beg=None, 
#     plot_t_end=None, 
#     draw_outage_limits_kwargs=None, 
#     include_outage_limits_text=False, 
#     out_t_beg_line_color='red', 
#     out_t_end_line_color='green', 
# ):
#     r"""
#     plot_t_beg/plot_t_end:
#         If supplied, out_t_beg and out_t_end will only be drawn if within plot_t_beg/plot_t_end interval
#     """
#     #----------------------------------------------------------------------------------------------------
#     if draw_outage_limits_kwargs is None:
#         draw_outage_limits_kwargs = {}    
#     #--------------------------------------------------
#     if plot_t_beg is None:
#         plot_t_beg = pd.Timestamp.min
#     if plot_t_end is None:
#         plot_t_end = pd.Timestamp.max
#     #--------------------------------------------------
#     # Draw the lines
#     if out_t_beg>=plot_t_beg and out_t_beg<=plot_t_end:
#         ax.axvline(out_t_beg, color=out_t_beg_line_color, **draw_outage_limits_kwargs)
#     if out_t_end>=plot_t_beg and out_t_end<=plot_t_end:
#         ax.axvline(out_t_end, color=out_t_end_line_color, **draw_outage_limits_kwargs)
#     #----------------------------------------------------------------------------------------------------
#     dflt_include_outage_limits_text_dict = dict(
#         include_out_t_beg_text = True, 
#         out_t_beg_text         = 'Outage Begin', 
#         out_t_beg_ypos         = ax.get_ylim()[0], 
#         out_t_beg_rot          = 90, 
#         out_t_beg_va           = 'bottom', 
#         out_t_beg_ha           = 'right', 
#         out_t_beg_color        = 'black', 
#         #----------
#         include_out_t_end_text = True, 
#         out_t_end_text         = 'Outage End', 
#         out_t_end_ypos         = ax.get_ylim()[0], 
#         out_t_end_rot          = 90, 
#         out_t_end_va           = 'bottom', 
#         out_t_end_ha           = 'left', 
#         out_t_end_color        = 'black', 
#     )
#     #-------------------------
#     if include_outage_limits_text:
#         if isinstance(include_outage_limits_text, dict):
#             include_outage_limits_text = Utilities.supplement_dict_with_default_values(
#                 to_supplmnt_dict    = include_outage_limits_text,
#                 default_values_dict = dflt_include_outage_limits_text_dict,
#                 extend_any_lists    = False,
#                 inplace             = False,
#             )
#         else:
#             include_outage_limits_text = dflt_include_outage_limits_text_dict
#         #-------------------------
#         include_out_t_beg_text = include_outage_limits_text['include_out_t_beg_text']
#         out_t_beg_text  = include_outage_limits_text['out_t_beg_text']
#         out_t_beg_ypos  = include_outage_limits_text['out_t_beg_ypos']
#         out_t_beg_rot   = include_outage_limits_text['out_t_beg_rot']
#         out_t_beg_va    = include_outage_limits_text['out_t_beg_va']
#         out_t_beg_ha    = include_outage_limits_text['out_t_beg_ha']
#         out_t_beg_color = include_outage_limits_text['out_t_beg_color']
#         #----------
#         include_out_t_end_text = include_outage_limits_text['include_out_t_end_text']
#         out_t_end_text  = include_outage_limits_text['out_t_end_text']
#         out_t_end_ypos  = include_outage_limits_text['out_t_end_ypos']
#         out_t_end_rot   = include_outage_limits_text['out_t_end_rot']
#         out_t_end_va    = include_outage_limits_text['out_t_end_va']
#         out_t_end_ha    = include_outage_limits_text['out_t_end_ha']
#         out_t_end_color = include_outage_limits_text['out_t_end_color']
#         #-------------------------
#         # If out_t_beg/out_t_end outside of plot_t_beg/plot_t_end, do not include the text, regardless
#         #   of the values of include_out_t_beg_text/include_out_t_end_text
#         if out_t_beg<plot_t_beg or out_t_beg>plot_t_end:
#             include_out_t_beg_text=False
#         if out_t_end<plot_t_beg or out_t_end>plot_t_end:
#             include_out_t_end_text=False
#         #-------------------------
#         if Utilities.is_object_one_of_types(out_t_beg_ypos, [list, tuple]) and len(out_t_beg_ypos)==2:
#             assert(out_t_beg_ypos[1] in ['eval_at_rt', 'ax_coord'])
#             if out_t_beg_ypos[1]=='eval_at_rt':
#                 out_t_beg_ypos = eval(out_t_beg_ypos[0])
#             else:
#                 out_t_beg_ypos = ax.transLimits.inverted().transform((0,out_t_beg_ypos[0]))[1]
#         #-----
#         if Utilities.is_object_one_of_types(out_t_end_ypos, [list, tuple]) and len(out_t_end_ypos)==2:
#             assert(out_t_end_ypos[1] in ['eval_at_rt', 'ax_coord'])
#             if out_t_end_ypos[1]=='eval_at_rt':
#                 out_t_end_ypos = eval(out_t_end_ypos[0])
#             else:
#                 out_t_end_ypos = ax.transLimits.inverted().transform((0,out_t_end_ypos[0]))[1]            
#         #-------------------------
#         if include_out_t_beg_text:
#             ax.text(
#                 x                   = out_t_beg, 
#                 y                   = out_t_beg_ypos, 
#                 s                   = out_t_beg_text, 
#                 rotation            = out_t_beg_rot, 
#                 verticalalignment   = out_t_beg_va, 
#                 horizontalalignment = out_t_beg_ha, 
#                 color               = out_t_beg_color
#             )
#         if include_out_t_end_text:
#             ax.text(
#                 x                   = out_t_end, 
#                 y                   = out_t_end_ypos, 
#                 s                   = out_t_end_text, 
#                 rotation            = out_t_end_rot, 
#                 verticalalignment   = out_t_end_va, 
#                 horizontalalignment = out_t_end_ha, 
#                 color               = out_t_end_color
#             )
#     #-------------------------
#     return ax

In [ ]:
Utilities.supplement_dict_with_default_values?

In [ ]:
ax = axs[3]

In [ ]:
ax.get_xlim()

In [ ]:
pd.Timestamp.min

In [ ]:
fig_num=0
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = means_df, 
    outg_rec_nb          = audit_i.outg_rec_nb, 
    outage_nb            = audit_i.outage_nb, 
    n_PNs_dovs           = audit_i.n_PNs_dovs, 
    ci_dovs              = audit_i.ci_cmi_dovs[0], 
    cmi_dovs             = audit_i.ci_cmi_dovs[0], 
    ci_ami               = audit_i.ci, 
    cmi_ami              = audit_i.cmi, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

# fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
#     n_PNs_w_power_srs, 
#     simp_freq='1T', 
#     threshold=threshold, 
#     fig_num=fig_num, 
#     fig_ax=(fig, axs[3]), 
#     threshold_color='magenta'
# )

for ax_i in axs:
    ax_i.xaxis.set_tick_params(labelbottom=True)

fig_num += 1

In [ ]:
    fig_num              = fig_num
    ami_df_i             = ami_df_i
    ami_df_i_out         = ami_df_i_out
    ami_df_i_not_out     = ami_df_i_not_out
    dovs_outg_t_beg      = dovs_outg_t_beg
    dovs_outg_t_end      = dovs_outg_t_end
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min()
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max()
    means_df             = None
    outg_rec_nb          = audit_i.outg_rec_nb
    outage_nb            = audit_i.outage_nb
    n_PNs_dovs           = audit_i.n_PNs_dovs
    ci_dovs              = audit_i.ci_cmi_dovs[0]
    cmi_dovs             = audit_i.ci_cmi_dovs[0]
    ci_ami               = audit_i.ci
    cmi_ami              = audit_i.cmi
    name                 = 'AMI'
    results_2_dict       = None
    expand_time          = pd.Timedelta('1 hour')
    mean_keys_to_include = ['winner', 'conservative', 'zero_times']
    removed_due_to_overlap_col=None
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True) 
    leg_i_plot=1
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2))
    ci_info_fontsize=16
    left_text_x=0.915

In [ ]:
only_connect_continuous=True
data_freq=pd.Timedelta('15T')

In [ ]:
if removed_due_to_overlap_col is not None:
    assert(removed_due_to_overlap_col in ami_df_i.columns.tolist())
    assert(removed_due_to_overlap_col in ami_df_i_out.columns.tolist())
    assert(removed_due_to_overlap_col in ami_df_i_not_out.columns.tolist())
#-------------------------
dflt_default_subplots_args = dict(
    n_x=1,
    n_y=3,
    fig_num=fig_num,
    sharex=False,
    sharey=False,
    unit_figsize_width=14,
    unit_figsize_height=6, 
    return_flattened_axes=True,
    row_major=True
)
if default_subplots_args is None:
    default_subplots_args = dflt_default_subplots_args
else:
    assert(isinstance(default_subplots_args, dict))
    default_subplots_args = Utilities.supplement_dict_with_default_values(
        to_supplmnt_dict=default_subplots_args, 
        default_values_dict=dflt_default_subplots_args, 
        extend_any_lists=False, 
        inplace=False
    )
fig, axs = Plot_General.default_subplots(**default_subplots_args)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))

palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)

#-------------------------
# The subplots share many common arguments.  
# Let's collect them all first to avoid repeating code
shared_plot_kwargs = dict(
    x                          = 'starttimeperiod_local', 
    y                          = 'value', 
    hue                        = 'serialnumber', 
    out_t_beg                  = dovs_outg_t_beg, 
    out_t_end                  = dovs_outg_t_end, 
    expand_time                = expand_time, 
    plot_time_beg_end          = [cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
    only_connect_continuous    = only_connect_continuous, 
    data_freq                  = data_freq,     
    data_label                 = '', 
    ax_args                    = None, 
    xlabel_args                = None, 
    ylabel_args                = None, 
    df_mean                    = None, 
    df_mean_col                = None, 
    mean_args                  = None, 
    draw_outage_limits         = True, 
    draw_outage_limits_kwargs  = dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
    include_outage_limits_text = dict(
        out_t_beg_text  = 'DOVS Beg.', 
        out_t_beg_ypos  = (0.12, 'ax_coord'), 
        out_t_beg_va    = 'bottom', 
        out_t_beg_ha    = 'center', 
        out_t_beg_color = 'red', 
        #-----
        out_t_end_text  = 'DOVS End', 
        out_t_end_ypos  = (0.12, 'ax_coord'), 
        out_t_end_va    = 'bottom', 
        out_t_end_ha    = 'center', 
        out_t_end_color = 'green', 
    ), 
    draw_without_hue_also      = False, 
    seg_line_freq              = None, 
    palette                    = palette
)


i_subplot=0
# fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
#     fig=fig, 
#     ax=axs[i_subplot], 
#     data=ami_df_i, 
#     title_args=dict(label=f"All (#SNs = {ami_df_i['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
#     **shared_plot_kwargs
# )


# AMINonVee.plot_usage(
#     fig=fig, 
#     ax=axs[i_subplot], 
#     data=ami_df_i, 
#     x='starttimeperiod_local', 
#     y='value', 
#     hue='serialnumber'
# )

# AMINonVee.plot_usage_around_outage(
#     fig=fig, 
#     ax=axs[i_subplot], 
#     data=ami_df_i, 
#     x='starttimeperiod_local', 
#     y='value', 
#     hue='serialnumber', 
#     out_t_beg                  = dovs_outg_t_beg, 
#     out_t_end                  = dovs_outg_t_end, 
#     expand_time                = expand_time, 
# )

# AMINonVee.plot_usage_around_outage(
#     fig=fig, 
#     ax=axs[i_subplot], 
#     data=ami_df_i, 
#     x='starttimeperiod_local', 
#     y='value', 
#     hue='serialnumber', 
#     out_t_beg                  = dovs_outg_t_beg, 
#     out_t_end                  = dovs_outg_t_end, 
#     expand_time                = expand_time, 
# )


fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=axs[i_subplot], 
    data=ami_df_i, 
    title_args=dict(label=f"All (#SNs = {ami_df_i['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
    **shared_plot_kwargs
)

In [ ]:
cnsrvtv_out_t_beg

In [ ]:
cnsrvtv_out_t_end

In [ ]:
ami_df_i

In [ ]:
fig,ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber'
)

In [ ]:
ami_df_i

In [ ]:
audit_i=copy.deepcopy(audit_1)

In [ ]:
ami_df_i = audit_i.ami_df_i.copy()
# In ami_df_i, mark any entries which were essentially removed via the identify_dovs_overlaps_from_best_ests
#   and removal procedure above
ami_df_i = DOVSAudit.set_removed_due_to_overlap_in_ami_df_i(
    ami_df_i                   = ami_df_i, 
    best_ests_df               = audit_i.best_ests_df_w_keep_info.copy(), 
    PN_col                     = 'aep_premise_nb', 
    time_idfr                  = 'starttimeperiod_local', 
    PN_col_be                  = 'PN', 
    keep_col_be                = 'keep', 
    overlap_times_col_be       = 'overlap_times', 
    removed_due_to_overlap_col = 'removed_due_to_overlap'
)

In [ ]:
best_ests_df = audit_i.best_ests_df_w_keep_info.copy()
ami_df_i = audit_i.ami_df_i.copy()
PN_col='aep_premise_nb'
time_idfr='starttimeperiod_local'
PN_col_be='PN'
t_min_col_be='winner_min'
t_max_col_be='winner_max'
keep_col_be='keep'
overlap_times_col_be='overlap_times'
removed_due_to_overlap_col='removed_due_to_overlap'
expand_removed_times=True

In [ ]:
assert(set([PN_col_be, keep_col_be, overlap_times_col_be]).difference(set(best_ests_df.columns.tolist()))==set())
removed_srs = best_ests_df[best_ests_df[keep_col_be]==False].copy()
removed_srs = removed_srs.groupby(PN_col_be)[overlap_times_col_be].sum()

In [ ]:
removed_srs

In [ ]:
best_ests_df[best_ests_df['PN']=='070171942']

In [ ]:
best_ests_df.iloc[:10]

In [ ]:
audit_1.best_ests_df_w_keep_info['ovrlp_disagree_typeA'].notna().any()

In [ ]:
update_df = audit_1.best_ests_df_w_keep_info.copy()
overlap_disagree_cols=['ovrlp_disagree_typeA', 'ovrlp_disagree_typeB']
unq_idfr_cols = ['PN', 'i_outg']

In [ ]:
update_df[overlap_disagree_cols[1]].notna().any()

In [ ]:
update_df[overlap_disagree_cols].notna().any().any()

In [ ]:
disagree_col = overlap_disagree_cols[1]

In [ ]:
update_df[disagree_col].notna().any()

In [ ]:
update_df[update_df[disagree_col].notna()]

In [ ]:
update_df[update_df[disagree_col].notna()][unq_idfr_cols].drop_duplicates().values.tolist()

In [ ]:
audit_1.best_ests_df_w_keep_info['ovrlp_disagree_typeA'].notna().sum()

In [ ]:
audit_1.best_ests_df_w_keep_info['ovrlp_disagree_typeB'].notna().sum()

In [ ]:
audit_1.best_ests_df_w_keep_info[
    (audit_1.best_ests_df_w_keep_info['ovrlp_disagree_typeA'].notna()) | 
    (audit_1.best_ests_df_w_keep_info['ovrlp_disagree_typeB'].notna())
]

In [ ]:
be_df_i = audit_1.best_ests_df_w_keep_info.copy()
outg_rec_nb_i = audit_1.outg_rec_nb
ovrlp_pfx = 'mXqy_'
overlap_DOVS_col = 'overlap_DOVS'

In [ ]:
regex_pattern  = r'{}.*'.format(ovrlp_pfx)
ovrlp_pct_cols = Utilities_df.find_cols_with_regex(df=be_df_i, regex_pattern=regex_pattern, ignore_case=False)
#-------------------------
outg_rec_nbs = [x[len(ovrlp_pfx):] for x in ovrlp_pct_cols]
if outg_rec_nb_i not in outg_rec_nbs:
    print(f'{outg_rec_nb_i} not in {outg_rec_nbs}!!!!!')
    assert(0)

In [ ]:
for idx_ij, row_ij in be_df_i.iterrows():
    print(idx_ij)
    DOVSAudit.assert_overlap_DOVS_col_agreement_helper(
        be_df_i_j=row_ij,
        outg_rec_nb_i=outg_rec_nb_i, 
        ovrlp_pfx=ovrlp_pfx, 
        ovrlp_pct_cols=ovrlp_pct_cols, 
        overlap_DOVS_col=overlap_DOVS_col
    )

In [ ]:
be_df_i_j[nonzero_cols]

In [ ]:
be_df_i_j[zero_cols].loc[be_df_i_j[zero_cols]>0].index.tolist()

In [ ]:
(be_df_i_j[zero_cols]==0)

In [ ]:
srs = pd.Series([0, 1, 0, 1])

In [ ]:
srs==0

In [ ]:
audit_1.best_ests_df_w_keep_info.iloc[7]

In [ ]:
be_df_i_j = audit_1.best_ests_df_w_keep_info.iloc[7].copy()

In [ ]:
assert(isinstance(be_df_i_j, pd.Series))
#-------------------------
ovrlp_DOVS = be_df_i_j[overlap_DOVS_col]
#-------------------------
nonzero_cols = [f'{ovrlp_pfx}{x}' for x in ovrlp_DOVS]
zero_cols = [x for x in ovrlp_pct_cols if x not in nonzero_cols+[f'{ovrlp_pfx}{outg_rec_nb_i}']]

In [ ]:
nonzero_cols

In [ ]:
zero_cols

In [ ]:
outg_rec_nb_i

In [ ]:
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
audit_2 = DOVSAudit(
#     outg_rec_nb='13464904'
    outg_rec_nb='13582332'
)
#-------------------------
audit_2.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
audit_2.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
audit_2.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)
#-------------------------
audit_2.build_best_ests_df()
#-------------------------


In [ ]:
audit_2.dovs_outg_t_beg_end

In [ ]:
audit_1.best_ests_df_w_keep_info.iloc[7]

In [ ]:
audit_1.best_ests_df_w_keep_info.iloc[:10]

In [ ]:
set(audit_1.best_ests_df_w_keep_info['overlap_DOVS'].sum())

In [ ]:
offenders = DOVSAudit.remove_prefix(inpt=offenders, prefix = ovrlp_pfx)

In [ ]:
audit_2.dovs_df_i.iloc[0]['premise_nbs']

In [ ]:
audit_2.ci_cmi_dovs

In [ ]:
audit_1.outg_rec_nb

In [ ]:
# Should flag any events with entries satisfying the following:
audit_1.best_ests_df_w_keep_info[
    (audit_1.best_ests_df_w_keep_info['resolved']==True) & 
    (audit_1.best_ests_df_w_keep_info['resolved_details']!='Certain')
]

In [ ]:
Utilities.find_idxs_in_list_with_regex(lst=['qe\/_13463901'], regex_pattern=r'qe\/_.*')

In [ ]:
FUCK

In [ ]:
# search_time_half_window=pd.Timedelta('24 hours')
# #-------------------------
# audit_2 = DOVSAudit(
#     outg_rec_nb='13464322'
# )
# #-------------------------
# audit_2.load_dovs(
#     dovs_df           = None, 
#     dovs_df_info_dict = None
# )
# #-------------------------
# audit_2.build_mp_df(
#     drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
# )
# #-------------------------
# audit_2.run_ami_and_ede_daq(
#     search_time_half_window=search_time_half_window
# )
# #-------------------------
# audit_2.build_best_ests_df()
# #-------------------------
# audit_2.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)
# #-------------------------
# resolve_overlapping_audits_NEW(audit_a=audit_2)

In [ ]:
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
audit_3 = DOVSAudit(
    outg_rec_nb='13464549'
)
#-------------------------
audit_3.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
audit_3.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
audit_3.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)
#-------------------------
audit_3.build_best_ests_df()
#-------------------------
audit_3.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)
#-------------------------
resolve_overlapping_audits_NEW(audit_a=audit_3)

In [ ]:
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
audit_4 = DOVSAudit(
    outg_rec_nb='13465188'
)
#-------------------------
audit_4.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
audit_4.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
audit_4.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)
#-------------------------
audit_4.build_best_ests_df()
#-------------------------
audit_4.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)
#-------------------------
resolve_overlapping_audits_NEW(audit_a=audit_4)

In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=0)
fig,ax = AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i[
        (ami_df_i['starttimeperiod_local']>pd.to_datetime('2023-07-03 12:00:00')) & 
        (ami_df_i['starttimeperiod_local']<pd.to_datetime('2023-07-04 14:00:00'))
    ], 
    x='starttimeperiod_local', 
    y='value', 
    hue='aep_premise_nb'
)
ax.legend().set_visible(False)
#-----
ax.axvline(pd.to_datetime('2023-07-03 15:34:00'), color='lightblue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='lightblue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 16:09:00'), color='blue', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 19:07:00'), color='blue', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-03 19:50:00'), color='green', ls='--')
ax.axvline(pd.to_datetime('2023-07-03 21:11:00'), color='green', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 04:04:00'), color='red', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 07:00:00'), color='red', ls='--')
#-----
ax.axvline(pd.to_datetime('2023-07-04 09:07:00'), color='orange', ls='--')
ax.axvline(pd.to_datetime('2023-07-04 11:30:00'), color='orange', ls='--')

In [ ]:
audit_1b, fig, ax = demo_func(
    outg_rec_nb='13464904', 
    search_time_half_window=pd.Timedelta('24 hours')
)

In [ ]:
audit_3b, fig, ax = demo_func(
    outg_rec_nb='13464549', 
    search_time_half_window=pd.Timedelta('24 hours')
)

In [ ]:
audit_4b, fig, ax = demo_func(
    outg_rec_nb='13465188', 
    search_time_half_window=pd.Timedelta('24 hours')
)

In [ ]:
# RULES:
#   - If a sub-outage only overlaps with one DOVS event, it is decided
#   - If a sub-outage does not overlap with any of the DOVS events, it remains with audit_a
#   - If a sub-outage does not overlap with audit_a, but does with others, remove from audit_i

In [ ]:
# PN_i
unq_idfr_cols = ['PN', 'i_outg']
PN_col='PN'
t_min_col='winner_min'
t_max_col='winner_max'
keep_col='keep'
open_beg_col='open_beg'
open_end_col='open_end'
overlap_dovs_col = 'overlap_DOVS'

outg_rec_nb_col_dovs='OUTG_REC_NB'
dt_off_ts_full_col_dovs='DT_OFF_TS_FULL'
dt_on_ts_col_dovs='DT_ON_TS'

In [ ]:
be_df_a

In [ ]:
list(set(be_df_a['overlap_DOVS'].sum()))

In [ ]:
list(set(be_df_a['overlap_times'].sum()))

In [ ]:
dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        premise_nbs=PN_i, 
        outg_rec_nbs=list(set(be_df_a['overlap_DOVS'].sum()))+[audit_a.outg_rec_nb], 
        include_premise=True
    ), 
    build_consolidated=False
)
dovs_df = dovs.df.copy()

In [ ]:
dovs_df

In [ ]:
be_df_a

In [ ]:
audit_a.best_ests_df_w_keep_info

In [ ]:
# PN_i
unq_idfr_cols = ['PN', 'i_outg']
PN_col='PN'
t_min_col='winner_min'
t_max_col='winner_max'
keep_col='keep'
open_beg_col='open_beg'
open_end_col='open_end'
overlap_dovs_col = 'overlap_DOVS'

outg_rec_nb_col_dovs='OUTG_REC_NB'
dt_off_ts_full_col_dovs='DT_OFF_TS_FULL'
dt_on_ts_col_dovs='DT_ON_TS'

In [ ]:
update_df = DOVSAudit.set_initial_to_adjust_df(
    be_df       = audit_a.best_ests_df_w_keep_info.copy(), 
    be_df_cols  = None, 
    addtnl_cols = None
)
#----------------------------------------------------------------------------------------------------
outg_rec_nbs = list(set(update_df[overlap_dovs_col].sum()))+[audit_a.outg_rec_nb]
#-------------------------
if(
    dovs_df is None or 
    set(outg_rec_nbs).difference(set(dovs_df[outg_rec_nb_col_dovs].unique().tolist())) != set()
):
    dovs = DOVSOutages(
        df_construct_type=DFConstructType.kRunSqlQuery, 
        contstruct_df_args=None, 
        init_df_in_constructor=True,
        build_sql_function=DOVSOutages_SQL.build_sql_outage, 
        build_sql_function_kwargs=dict(
            premise_nbs=PN_i, 
            outg_rec_nbs=outg_rec_nbs, 
            include_premise=True
        ), 
        build_consolidated=False
    )
    dovs_df = dovs.df.copy()
#-------------------------
# If dovs_df supplied, don't want to make changes outside of this function, so copy
dovs_df = dovs_df.copy()
assert(set(outg_rec_nbs).difference(set(dovs_df[outg_rec_nb_col_dovs].unique().tolist()))==set())
#-----
dovs_df = dovs_df[dovs_df[outg_rec_nb_col_dovs].isin(outg_rec_nbs)]
dovs_df = dovs_df[[outg_rec_nb_col_dovs, dt_off_ts_full_col_dovs, dt_on_ts_col_dovs]].drop_duplicates()
#-----
# There should only be one entry in dovs_df for each outg_rec_nb
assert(dovs_df[outg_rec_nb_col_dovs].nunique()==dovs_df.shape[0])
#----------------------------------------------------------------------------------------------------
# Build new columns, one for each outg_rec_nb, housing the percent overlap of the found interval for each row
#   and the possibly overlapping DOVS events
#-----
ovrlp_pfx = Utilities.generate_random_string(str_len=4)+'_'
assert(not col.startswith(ovrlp_pfx) for col in update_df.columns.tolist())
#-----
for idx_i, dovs_i in dovs_df.iterrows():
    outg_rec_nb_i = dovs_i[outg_rec_nb_col_dovs]
    update_df[f'{ovrlp_pfx}{outg_rec_nb_i}'] = update_df.apply(
        lambda x: DOVSAudit.get_overlap_interval_len(
            intrvl_1 = [x[t_min_col], x[t_max_col]], 
            intrvl_2 = [dovs_i[dt_off_ts_full_col_dovs], dovs_i[dt_on_ts_col_dovs]], 
            norm_by=2
        ), 
        axis=1
    )
#-------------------------
ovrlp_pct_cols = [x for x in update_df.columns.tolist() if x.startswith(ovrlp_pfx)]
assert(len(ovrlp_pct_cols)==len(outg_rec_nbs))

#----------------------------------------------------------------------------------------------------
# RULES:
#   - If a sub-outage only overlaps with one DOVS event, it is decided
#   - If a sub-outage does not overlap with any of the DOVS events, it remains with audit_a
#   - If a sub-outage does not overlap with audit_a, but does with others, remove from audit_i
#-------------------------
# There might be some clever, pythonic, way to achieve this (e.g., can definitely identify via calls
#   like (update_df[ovrlp_pct_cols]>0).sum(axis=1) and (update_df[ovrlp_pct_cols]==0).sum(axis=1)).
# But, the operations are light and simple, so brute-force is easier and fine in this case
#-------------------------
# Need unique indices, so I can safely assign using .loc
# If one did not want to do the .reset_index call below, one could iterate over rows using 
#   for i_row in range(update_df.shape[0]) in the for loop below
if update_df.index.nunique()<update_df.shape[0]:
    update_df = update_df.reset_index(drop=True)
#-----
for idx_i, row_i in update_df.iterrows():
    gt0_srs = row_i[ovrlp_pct_cols]>0
    eq0_srs = row_i[ovrlp_pct_cols]==0
    #-----
    # If a sub-outage only overlaps with one DOVS event, it is decided
    if gt0_srs.sum()==1:
        assert(len(row_i[overlap_dovs_col])<=1)
        parent_dovs = gt0_srs[gt0_srs==True].index
        assert(len(parent_dovs)==1)
        parent_dovs = remove_prefix(
            inpt   = parent_dovs[0], 
            prefix = ovrlp_pfx
        )
        #-------------------------
        update_df.loc[idx_i, 'resolved']         = True
        update_df.loc[idx_i, 'resolved_details'] = 'Certain'
        if parent_dovs == audit_a.outg_rec_nb:
            update_df.loc[idx_i, 'adjustment']   = None
        else:
            update_df.loc[idx_i, 'adjustment']   = 'remove'
    # If a sub-outage does not overlap with any of the DOVS events, it remains with audit_a
    elif eq0_srs.all():
        assert(len(row_i[overlap_dovs_col])==0)
        update_df.loc[idx_i, 'resolved']         = True
        update_df.loc[idx_i, 'resolved_details'] = 'Certain'
        update_df.loc[idx_i, 'adjustment']       = None
    # Sub-outage overlaps with multipl DOVS events......
    else:
        assert(gt0_srs.sum()>1)
        assert(
            len(row_i[overlap_dovs_col])==gt0_srs.sum() or 
            len(row_i[overlap_dovs_col])==gt0_srs.sum()-1
        )
        #-------------------------
        ovrlp_outg_rec_nbs = remove_prefix(
            inpt   = gt0_srs[gt0_srs==True].index.tolist(), 
            prefix = ovrlp_pfx
        )
        #-------------------------
        # ovrlp_outg_rec_nbs should contain all entries from row_i[overlap_dovs_col] and possibly audit_a.outg_rec_nb
        assert(set(row_i[overlap_dovs_col]).difference(set(ovrlp_outg_rec_nbs))==set())
        #-------------------------
        # If audit_a.outg_rec_nb does not overlap with the sub-outage, no further analysis is needed,
        #   this sub-outage simply needs to be removed from audit_i
        if audit_a.outg_rec_nb not in ovrlp_outg_rec_nbs:
            update_df.loc[idx_i, 'resolved']         = True
            update_df.loc[idx_i, 'resolved_details'] = 'Certain'
            update_df.loc[idx_i, 'adjustment']       = 'remove'
        # If audit_a.outg_rec_nb does overlap sub-outage (with others), split using
        #   split_outage_interval_across_multiple_overlapping_dovs
        else:
            outg_intrvl = [row_i[t_min_col], row_i[t_max_col]]
            ovrlp_outg_rec_nbs = remove_prefix(
                inpt   = gt0_srs[gt0_srs==True].index.tolist(), 
                prefix = ovrlp_pfx
            )
            split_df = split_outage_interval_across_multiple_overlapping_dovs(
                outg_intrvl = outg_intrvl, 
                outg_rec_nbs = ovrlp_outg_rec_nbs, 
                dovs_df=None, 
                outg_rec_nb_col='OUTG_REC_NB', 
                dt_off_ts_full_col='DT_OFF_TS_FULL', 
                dt_on_ts_col='DT_ON_TS'
            )
            #-------------------------
            assert(audit_a.outg_rec_nb in split_df['OUTG_REC_NB'].unique().tolist())
            adjstmnt_i = split_df[split_df['OUTG_REC_NB']==audit_a.outg_rec_nb]
            assert(adjstmnt_i.shape[0]==1)
            #-----
            update_df.loc[idx_i, 'resolved']         = True
            update_df.loc[idx_i, 'resolved_details'] = 'Uncertain'
            update_df.loc[idx_i, 'adjustment']       = 'adjust'
            #-----
            update_df.loc[idx_i, t_min_col] = adjstmnt_i.iloc[0]['intrvl_min']
            update_df.loc[idx_i, t_max_col] = adjstmnt_i.iloc[0]['intrvl_max']
#-------------------------
update_df = update_df.drop(columns=ovrlp_pct_cols)

In [ ]:
update_df

In [ ]:
update_df[update_df['keep']==True]

In [ ]:
update_df[update_df['keep']!=True]

In [ ]:
update_df[update_df['adjustment']!='remove']

In [ ]:
be_df_cols = unq_idfr_cols + list(set([t_min_col, t_max_col, open_beg_col, open_end_col, overlap_dovs_col]).difference(set(unq_idfr_cols)))
assert(set(be_df_cols).difference(set(be_df_a.columns.tolist()))==set())
return_cols = be_df_cols + ['adjustment', 'resolved', 'resolved_details']
#-------------------------
update_df = DOVSAudit.set_initial_to_adjust_df(
    be_df       = be_df_a.copy(), 
    be_df_cols  = be_df_cols, 
    addtnl_cols = None
)
#----------------------------------------------------------------------------------------------------
outg_rec_nbs = list(set(update_df[overlap_dovs_col].sum()))+[audit_a.outg_rec_nb]
#-------------------------
if(
    dovs_df is None or 
    set(outg_rec_nbs).difference(set(dovs_df[outg_rec_nb_col_dovs].unique().tolist())) != set()
):
    dovs = DOVSOutages(
        df_construct_type=DFConstructType.kRunSqlQuery, 
        contstruct_df_args=None, 
        init_df_in_constructor=True,
        build_sql_function=DOVSOutages_SQL.build_sql_outage, 
        build_sql_function_kwargs=dict(
            premise_nbs=PN_i, 
            outg_rec_nbs=outg_rec_nbs, 
            include_premise=True
        ), 
        build_consolidated=False
    )
    dovs_df = dovs.df.copy()
#-------------------------
# If dovs_df supplied, don't want to make changes outside of this function, so copy
dovs_df = dovs_df.copy()
assert(set(outg_rec_nbs).difference(set(dovs_df[outg_rec_nb_col_dovs].unique().tolist()))==set())
#-----
dovs_df = dovs_df[dovs_df[outg_rec_nb_col_dovs].isin(outg_rec_nbs)]
dovs_df = dovs_df[[outg_rec_nb_col_dovs, dt_off_ts_full_col_dovs, dt_on_ts_col_dovs]].drop_duplicates()
#-----
# There should only be one entry in dovs_df for each outg_rec_nb
assert(dovs_df[outg_rec_nb_col_dovs].nunique()==dovs_df.shape[0])
#----------------------------------------------------------------------------------------------------
# Build new columns, one for each outg_rec_nb, housing the percent overlap of the found interval for each row
#   and the possibly overlapping DOVS events
#-----
ovrlp_pfx = Utilities.generate_random_string(str_len=4)+'_'
assert(not col.startswith(ovrlp_pfx) for col in update_df.columns.tolist())
#-----
for idx_i, dovs_i in dovs_df.iterrows():
    outg_rec_nb_i = dovs_i[outg_rec_nb_col_dovs]
    update_df[f'{ovrlp_pfx}{outg_rec_nb_i}'] = update_df.apply(
        lambda x: DOVSAudit.get_overlap_interval_len(
            intrvl_1 = [x[t_min_col], x[t_max_col]], 
            intrvl_2 = [dovs_i[dt_off_ts_full_col_dovs], dovs_i[dt_on_ts_col_dovs]], 
            norm_by=2
        ), 
        axis=1
    )
#-------------------------
ovrlp_pct_cols = [x for x in update_df.columns.tolist() if x.startswith(ovrlp_pfx)]
assert(len(ovrlp_pct_cols)==len(outg_rec_nbs))

#----------------------------------------------------------------------------------------------------
# RULES:
#   - If a sub-outage only overlaps with one DOVS event, it is decided
#   - If a sub-outage does not overlap with any of the DOVS events, it remains with audit_a
#   - If a sub-outage does not overlap with audit_a, but does with others, remove from audit_i
#-------------------------
# There might be some clever, pythonic, way to achieve this (e.g., can definitely identify via calls
#   like (update_df[ovrlp_pct_cols]>0).sum(axis=1) and (update_df[ovrlp_pct_cols]==0).sum(axis=1)).
# But, the operations are light and simple, so brute-force is easier and fine in this case
#-------------------------
# Need unique indices, so I can safely assign using .loc
# If one did not want to do the .reset_index call below, one could iterate over rows using 
#   for i_row in range(update_df.shape[0]) in the for loop below
if update_df.index.nunique()<update_df.shape[0]:
    update_df = update_df.reset_index(drop=True)
#-----
for idx_i, row_i in update_df.iterrows():
    gt0_srs = row_i[ovrlp_pct_cols]>0
    eq0_srs = row_i[ovrlp_pct_cols]==0
    #-----
    # If a sub-outage only overlaps with one DOVS event, it is decided
    if gt0_srs.sum()==1:
        assert(len(row_i[overlap_dovs_col])<=1)
        parent_dovs = gt0_srs[gt0_srs==True].index
        assert(len(parent_dovs)==1)
        parent_dovs = remove_prefix(
            inpt   = parent_dovs[0], 
            prefix = ovrlp_pfx
        )
        #-------------------------
        update_df.loc[idx_i, 'resolved']         = True
        update_df.loc[idx_i, 'resolved_details'] = 'Certain'
        if parent_dovs == audit_a.outg_rec_nb:
            update_df.loc[idx_i, 'adjustment']   = None
        else:
            update_df.loc[idx_i, 'adjustment']   = 'remove'
    # If a sub-outage does not overlap with any of the DOVS events, it remains with audit_a
    elif eq0_srs.all():
        assert(len(row_i[overlap_dovs_col])==0)
        update_df.loc[idx_i, 'resolved']         = True
        update_df.loc[idx_i, 'resolved_details'] = 'Certain'
        update_df.loc[idx_i, 'adjustment']       = None
    # Sub-outage overlaps with multipl DOVS events......
    else:
        assert(gt0_srs.sum()>1)
        assert(
            len(row_i[overlap_dovs_col])==gt0_srs.sum() or 
            len(row_i[overlap_dovs_col])==gt0_srs.sum()-1
        )
        #-------------------------
        ovrlp_outg_rec_nbs = remove_prefix(
            inpt   = gt0_srs[gt0_srs==True].index.tolist(), 
            prefix = ovrlp_pfx
        )
        #-------------------------
        # ovrlp_outg_rec_nbs should contain all entries from row_i[overlap_dovs_col] and possibly audit_a.outg_rec_nb
        assert(set(row_i[overlap_dovs_col]).difference(set(ovrlp_outg_rec_nbs))==set())
        #-------------------------
        # If audit_a.outg_rec_nb does not overlap with the sub-outage, no further analysis is needed,
        #   this sub-outage simply needs to be removed from audit_i
        if audit_a.outg_rec_nb not in ovrlp_outg_rec_nbs:
            update_df.loc[idx_i, 'resolved']         = True
            update_df.loc[idx_i, 'resolved_details'] = 'Certain'
            update_df.loc[idx_i, 'adjustment']       = 'remove'
        # If audit_a.outg_rec_nb does overlap sub-outage (with others), split using
        #   split_outage_interval_across_multiple_overlapping_dovs
        else:
            outg_intrvl = [row_i[t_min_col], row_i[t_max_col]]
            ovrlp_outg_rec_nbs = remove_prefix(
                inpt   = gt0_srs[gt0_srs==True].index.tolist(), 
                prefix = ovrlp_pfx
            )
            split_df = split_outage_interval_across_multiple_overlapping_dovs(
                outg_intrvl = outg_intrvl, 
                outg_rec_nbs = ovrlp_outg_rec_nbs, 
                dovs_df=None, 
                outg_rec_nb_col='OUTG_REC_NB', 
                dt_off_ts_full_col='DT_OFF_TS_FULL', 
                dt_on_ts_col='DT_ON_TS'
            )
            #-------------------------
            assert(audit_a.outg_rec_nb in split_df['OUTG_REC_NB'].unique().tolist())
            adjstmnt_i = split_df[split_df['OUTG_REC_NB']==audit_a.outg_rec_nb]
            assert(adjstmnt_i.shape[0]==1)
            #-----
            update_df.loc[idx_i, 'resolved']         = True
            update_df.loc[idx_i, 'resolved_details'] = 'Uncertain'
            update_df.loc[idx_i, 'adjustment']       = 'adjust'
            #-----
            update_df.loc[idx_i, t_min_col] = adjstmnt_i.iloc[0]['intrvl_min']
            update_df.loc[idx_i, t_max_col] = adjstmnt_i.iloc[0]['intrvl_max']
#-------------------------
update_df = update_df.drop(columns=ovrlp_pct_cols)

In [ ]:
row_i = update_df.iloc[0]

In [ ]:
len(row_i[overlap_dovs_col])

In [ ]:
# IMPORTANT: Since resolved is initiated to np.nan, one cannot simply take 
#              audit_a1.best_ests_df_w_keep_info['resolved'].sum(), but must instead do the following
(audit_a1.best_ests_df_w_keep_info['resolved']==True).sum()

In [ ]:
dev_PNs = audit_i.best_ests_df['PN'].unique().tolist()

dovs_dev = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs=dict(
        premise_nbs=dev_PNs, 
        field_to_split='premise_nbs', 
        include_premise=True
    ), 
    build_consolidated=False
)
dovs_df_dev = dovs_dev.df.copy()

In [ ]:
natsorted(dovs_df_dev['DT_OFF_TS_FULL'].unique().tolist())

In [ ]:
audit_i.dovs_outg_t_beg_end

In [ ]:
print(dovs_dev.get_sql_statement())

In [ ]:
dovs_dev = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs=dict(
        premise_nbs=['964325483'], 
        field_to_split='premise_nbs', 
        include_premise=True
    ), 
    build_consolidated=False
)
dovs_df_dev = dovs_dev.df.copy()

In [ ]:
natsorted(dovs_df_dev['DT_OFF_TS_FULL'].unique().tolist())

In [ ]:
dovs_df_dev.sort_values(by=['DT_OFF_TS']).tail(10)

In [ ]:
audit_i.outg_rec_nb

In [ ]:
def remove_overlap_from_interval_w_split(
    interval, 
    overlap, 
    split_interval=False
):
    r"""
    Not really used anywhere.
    This is how remove_overlap_from_interval was originally written, but I reduced the functionality to
      simplify things.
    So, this exists in case I desire the functionality later.
    
    Given an input interval and possible overlap interval, remove overlap from the interval.
    Both interval and overlap should be lists/tuples of length 2 whose elements represent the beginning and
      ending of the interval.
    
    By default, this will not split the input interval.
        If the overlap does split the interval, the portion of interval occurring prior to the overlap will be returned.
        The intent of this function is to remove any DOVS events overlapping with the algorithm outage times.
        Therefore, in such a split situation, we would not want anything after the first overlap.
    --------------------------------------------------
    There are basically 4 distinct situations
      1. overlap completely encompasses interval 
           ==> returned interval should be empty
      2. interval completely encompasses overlap
           - if split_interval==True: 
                 return the two sub-intervals on either side of overlap
           - if split_interval==False: 
                 return the first sub-interval which occurs prior to overlap
      3. overlap begins BEFORE interval begins and ends BEFORE interval ends
           ==> return portion of interval occurring after overlap ends
      4. overlap begins AFTER interval begins and ends AFTER interval ends
           ==> return portion of interval occurring before overlap begins
    """
    #--------------------------------------------------
    assert(Utilities.is_object_one_of_types(interval, [list, tuple]))
    assert(Utilities.is_object_one_of_types(overlap, [list, tuple]))
    #-----
    if len(interval)==0:
        return interval
    #-----
    assert(len(interval)==2)
    assert(len(overlap)==2)
    #-----
    assert(interval[1] > interval[0])
    assert(overlap[1]  > overlap[0])
    #-------------------------
    # If the two do not overlap, simply return interval
    if not pd.Interval(interval[0], interval[1]).overlaps(pd.Interval(overlap[0], overlap[1])):
        return interval
    #--------------------------------------------------
    # 4 distinct situations described in function documentation
    #-------------------------
    # 1. overlap completely encompasses interval 
    if (overlap[0] <= interval[0]) and (overlap[1] >= interval[1]):
        return []
    #-------------------------
    # 2. interval completely encompasses overlap
    if (interval[0] <= overlap[0]) and (interval[1] >= overlap[1]):
        intrvl_1 = [interval[0], overlap[0]]
        intrvl_2 = [overlap[1],  interval[1]]
        if split_interval:
            return intrvl_1, intrvl_2
        else:
            return intrvl_1
        
    #-------------------------
    # 3. overlap begins BEFORE interval begins and ends BEFORE interval ends
    #     ==> return portion of interval occurring after overlap ends
    if overlap[0] < interval[0]:
        assert(overlap[1] < interval[1])
        return [overlap[1], interval[1]]
    
    #-------------------------
    # 4. overlap begins AFTER interval begins and ends AFTER interval ends
    #     ==> return portion of interval occurring before overlap begins
    if overlap[0] > interval[0]:
        assert(overlap[1] > interval[1])
        return [interval[0], overlap[0]]
    
    #-------------------------
    # SHOULD NEVER REACH THIS POINT!
    print('In remove_overlap_from_interval: None of 4 distinct situations caught current!\nCRASH IMMINENT!')
    print(f'interval = {interval}')
    print(f'overlap  = {overlap}')
    assert(0)

In [ ]:
def remove_overlap_from_interval(
    interval, 
    overlap, 
    return_removed=False
):
    r"""
    Given an input interval and possible overlap interval, remove overlap from the interval.
    Both interval and overlap should be lists/tuples of length 2 whose elements represent the beginning and
      ending of the interval.
    
    This will NOT split the input interval.
        If the overlap does split the interval, the portion of interval occurring prior to the overlap will be returned.
        The intent of this function is to remove any DOVS events overlapping with the algorithm outage times.
        Therefore, in such a split situation, we would not want anything after the first overlap.
    --------------------------------------------------
    There are basically 4 distinct situations
      1. overlap completely encompasses interval 
           ==> returned interval should be empty
      2. interval completely encompasses overlap
           ==> return the first sub-interval which occurs prior to overlap
      3. overlap begins BEFORE interval begins and ends BEFORE interval ends
           ==> return portion of interval occurring after overlap ends
      4. overlap begins AFTER interval begins and ends AFTER interval ends
           ==> return portion of interval occurring before overlap begins
    """
    #--------------------------------------------------
    assert(Utilities.is_object_one_of_types(interval, [list, tuple]))
    assert(Utilities.is_object_one_of_types(overlap, [list, tuple]))
    #-----
    if len(interval)==0:
        if return_removed:
            return interval, []
        return interval
    #-----
    assert(len(interval)==2)
    assert(len(overlap)==2)
    #-----
    assert(interval[1] > interval[0])
    assert(overlap[1]  > overlap[0])
    #-------------------------
    # If the two do not overlap, simply return interval
    if not pd.Interval(interval[0], interval[1]).overlaps(pd.Interval(overlap[0], overlap[1])):
        rtrn_intrvl = []
        rmvd_intrvl = []
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
    #--------------------------------------------------
    # 4 distinct situations described in function documentation
    #-------------------------
    # 1. overlap completely encompasses interval 
    if (overlap[0] <= interval[0]) and (overlap[1] >= interval[1]):
        rtrn_intrvl = []
        rmvd_intrvl = [overlap[0], overlap[1]]
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
    #-------------------------
    # 2. interval completely encompasses overlap
    if (interval[0] <= overlap[0]) and (interval[1] >= overlap[1]):
        rtrn_intrvl = [interval[0], overlap[0]]
        rmvd_intrvl = [overlap[0], interval[1]]
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
        
    #-------------------------
    # 3. overlap begins BEFORE interval begins and ends BEFORE interval ends
    #     ==> return portion of interval occurring after overlap ends
    if overlap[0] < interval[0]:
        assert(overlap[1] < interval[1])
        rtrn_intrvl = [overlap[1], interval[1]]
        rmvd_intrvl = [overlap[0], overlap[1]]
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
    
    #-------------------------
    # 4. overlap begins AFTER interval begins and ends AFTER interval ends
    #     ==> return portion of interval occurring before overlap begins
    if overlap[0] > interval[0]:
        assert(overlap[1] > interval[1])
        rtrn_intrvl = [interval[0], overlap[0]]
        rmvd_intrvl = [overlap[0], overlap[1]]
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
    
    #-------------------------
    # SHOULD NEVER REACH THIS POINT!
    print('In remove_overlap_from_interval: None of 4 distinct situations caught current!\nCRASH IMMINENT!')
    print(f'interval = {interval}')
    print(f'overlap  = {overlap}')
    assert(0)

In [ ]:
def remove_overlaps_from_interval(
    interval, 
    overlaps, 
    return_removed=False
):
    r"""
    Remove multiple possible overlaps from interval
    """
    #-------------------------
    assert(isinstance(overlaps, list))
    assert(Utilities.are_all_list_elements_one_of_types(lst=overlaps, types=[list, tuple]))
    assert(Utilities.are_list_elements_lengths_homogeneous(lst=overlaps, length=2))
    #-------------------------
    return_int = copy.deepcopy(interval)
    removed_intrvls = []
    for overlap_i in overlaps:
        if len(return_int)==0:
            break
        return_int, rmvd_intrvl_i = remove_overlap_from_interval(
            interval       = return_int, 
            overlap        = overlap_i, 
            return_removed = True
        )
        removed_intrvls.append(rmvd_intrvl_i)
    #-------------------------
    if return_removed:
        return return_int, removed_intrvls
    return return_int

In [ ]:
best_ests_df_w_keep_info = audit_i.best_ests_df_w_keep_info.copy()

In [ ]:
best_ests_df_w_keep_info

In [ ]:
# winner_min    = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False].iloc[0]['winner_min']
# winner_max    = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False].iloc[0]['winner_max']
# overlap_times = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False].iloc[0]['overlap_times']

winner_min    = best_ests_df_w_keep_info.iloc[3]['winner_min']
winner_max    = best_ests_df_w_keep_info.iloc[3]['winner_max']
overlap_times = best_ests_df_w_keep_info.iloc[3]['overlap_times']

print(winner_min)
print(winner_max)
print(overlap_times)

In [ ]:
# remove_overlap_from_interval(
#     interval = [winner_min, winner_max], 
#     overlap = overlap_times[0], 
#     return_removed=False
# )

In [ ]:
# remove_overlap_from_interval(
#     interval = [winner_min, winner_max], 
#     overlap = overlap_times[0], 
#     return_removed=True
# )

In [ ]:
# r1,r2 = remove_overlaps_from_interval(
#     interval = [winner_min, winner_max], 
#     overlaps = overlap_times, 
#     return_removed=True
# )
# print(r1)
# print(r2)

In [ ]:
ovrlp_audit_j.best_ests_df_w_keep_info

In [ ]:
# Easiest situation:
#   outg_rec_nb_a finds      two sub-outages, with sub_a_1 marked keep=True  and sub_a_2 marked keep=False
#   outg_rec_nb_b finds same two sub-outages, with sub_a_1 marked keep=False and sub_a_2 marked keep=True

In [ ]:
# Can't get around doing this PN-by-PN

In [ ]:
best_ests_df_w_keep_info = audit_i.best_ests_df_w_keep_info.copy()

In [ ]:
def remove_overlaps_for_PN_i(
    best_ests_df_i, 
    ovrlp_be_df, 
    t_min_col_idx, 
    t_max_col_idx, 
    rmvd_times_col_idx, 
    PN_col='PN', 
    keep_col='keep'
):
    r"""
    HELPER FUNCTION! Not designed to be used on its own
        
    In reality, best_ests_df_i should be best_ests_df_w_keep_info_i, but I didn't feel like carrying such a heavy
      name around the entire function
    """
    #-------------------------
    assert(best_ests_df_i[PN_col].nunique()==1)
    best_ests_df_i = best_ests_df_i.copy()
    #-------------------------
    PN_i = best_ests_df_i[PN_col].unique().tolist()[0]
    ovrlp_be_df_i = ovrlp_be_df[ovrlp_be_df[PN_col]==PN_i]
    if ovrlp_be_df_i.shape[0]==0:
        return best_ests_df_i
    #-------------------------
    t_min_col = best_ests_df_i.columns.tolist()[t_min_col_idx]
    t_max_col = best_ests_df_i.columns.tolist()[t_max_col_idx]
    #-------------------------
    # Annoying, but ovrlp_intervals_i needs to be a list of 2-d lists holding timestamp objects.
    #   Calling .values.tolist() converts the timestamps to ints, which is the annoying part.
    #   Therefore, I have to use pd.to_datetime() to convert back
    ovrlp_intervals_i = ovrlp_be_df_i[[t_min_col, t_max_col]].values
    ovrlp_intervals_i = [[pd.to_datetime(intrvl[0]), pd.to_datetime(intrvl[1])] for intrvl in ovrlp_intervals_i.tolist()]
    # Can't get around doing this row-by-row
    for idx_j in range(best_ests_df_i.shape[0]):
        #-------------------------
        if best_ests_df_i.iloc[idx_j][keep_col]==True:
            continue
        #-------------------------
        t_min_j = best_ests_df_i.iloc[idx_j][t_min_col]
        t_max_j = best_ests_df_i.iloc[idx_j][t_max_col]
        #-----
        interval_j, removed_j = remove_overlaps_from_interval(
            interval = [t_min_j, t_max_j], 
            overlaps = ovrlp_intervals_i, 
            return_removed=True
        )
        #-----
        if len(interval_j)==0:
            best_ests_df_i.iloc[idx_j, t_min_col_idx] = None
            best_ests_df_i.iloc[idx_j, t_max_col_idx] = None
        else:
            best_ests_df_i.iloc[idx_j, t_min_col_idx] = interval_j[0]
            best_ests_df_i.iloc[idx_j, t_max_col_idx] = interval_j[1]
        #-----
        # NOTE: Using iat instead of iloc.
        #       Using iloc gives "ValueError: Must have equal len keys and value when setting with an ndarray"
        #         because pandas thinks I am interacting with multiple rows
        #       Using iat resolves the confusion by letting pandas know I am dealing with a single element
        best_ests_df_i.iat[idx_j, rmvd_times_col_idx] = removed_j
    #-------------------------
    return best_ests_df_i

In [ ]:
# def remove_overlaps_from_best_ests(
#     best_ests_df, 
#     ovrlp_be_df, 
#     PN_col='PN', 
#     t_min_col='winner_min', 
#     t_max_col='winner_max', 
#     keep_col='keep'
# ):
#     r"""
#     In reality, best_ests_df should be best_ests_df_w_keep_info, but I didn't feel like carrying such a heavy
#       name around the entire function
#     """
#     #-------------------------

# TODO
 - Need to develop method for creating ovrlp_be_df from multiple ovrlp_audit_j
 - See note: !!!!!!!!!! May want ot save status of return_df right now to some class member in case need to examine later
 - CI_NB/CMI_NB > 0 as default

In [ ]:
best_ests_df = best_ests_df_w_keep_info.copy()
ovrlp_be_df = ovrlp_audit_j.best_ests_df_w_keep_info.copy()
PN_col='PN'
t_min_col='winner_min'
t_max_col='winner_max'
keep_col='keep'

In [ ]:
#--------------------------------------------------
t_min_OG_col = t_min_col+'_OG'
if t_min_OG_col in best_ests_df.columns.tolist():
    t_min_OG_col = t_min_OG_col+'_'+Utilities.generate_random_string(str_len=4)
#-----
t_max_OG_col = t_max_col+'_OG'
if t_max_OG_col in best_ests_df.columns.tolist():
    t_max_OG_col = t_max_OG_col+'_'+Utilities.generate_random_string(str_len=4)
#-----
removed_times_col = 'removed_times'
if removed_times_col in best_ests_df.columns.tolist():
    removed_times_col = removed_times_col+'_'+Utilities.generate_random_string(str_len=4)
#-----
assert(t_min_OG_col not in best_ests_df.columns.tolist())
assert(t_max_OG_col not in best_ests_df.columns.tolist())
assert(removed_times_col not in best_ests_df.columns.tolist())
#-------------------------
best_ests_df[t_min_OG_col]      = best_ests_df[t_min_col]
best_ests_df[t_max_OG_col]      = best_ests_df[t_max_col]
best_ests_df[removed_times_col] = [[] for _ in range(best_ests_df.shape[0])]
#--------------------------------------------------
t_min_col_idx = Utilities_df.find_idxs_in_highest_order_of_columns(
    df = best_ests_df, 
    col = 'winner_min', 
    exact_match=True, 
    assert_single=True
)
t_max_col_idx = Utilities_df.find_idxs_in_highest_order_of_columns(
    df = best_ests_df, 
    col = 'winner_max', 
    exact_match=True, 
    assert_single=True
)
rmvd_times_col_idx = Utilities_df.find_idxs_in_highest_order_of_columns(
    df = best_ests_df, 
    col = 'removed_times', 
    exact_match=True, 
    assert_single=True
)
#--------------------------------------------------
return_df = best_ests_df.groupby(PN_col, as_index=False, group_keys=False).apply(
    lambda x: remove_overlaps_for_PN_i(
        best_ests_df_i     = x, 
        ovrlp_be_df        = ovrlp_be_df, 
        t_min_col_idx      = t_min_col_idx, 
        t_max_col_idx      = t_max_col_idx, 
        rmvd_times_col_idx = rmvd_times_col_idx, 
        PN_col             = PN_col, 
        keep_col           = keep_col
    )
)
#!!!!!!!!!! May want ot save status of return_df right now to some class member in case need to examine later
#--------------------------------------------------
# First, call dropna with how='all'
#   This should handle all cases, but I would like to be notified if a sub-outage has a value for t_min_col
#     but not for t_max_col (or vice-versa)
return_df = return_df.dropna(subset=[t_min_col, t_max_col], how='all')
# NOTE: If one wanted to identify the rows for which one member is NA, one could use
#         return_df[[t_min_col, t_max_col]].isna().sum(axis=1)>0
assert(return_df[[t_min_col, t_max_col]].isna().sum().sum()==0)
#--------------------------------------------------
return_df = DOVSAudit.set_i_outg_in_best_ests_df(
    best_ests_df = return_df, 
    groupby_cols = [PN_col], 
    sort_cols    = [t_min_col, t_min_col], 
    i_outg_col   = 'i_outg'
)

In [ ]:
# First, call dropna with how='all'
#   This should handle all cases, but I would like to be notified if a sub-outage has a value for t_min_col
#     but not for t_max_col (or vice-versa)
return_df = return_df.dropna(subset=[t_min_col, t_max_col], how='all')
# NOTE: If one wanted to identify the rows for which one member is NA, one could use
#         return_df[[t_min_col, t_max_col]].isna().sum(axis=1)>0
assert(return_df[[t_min_col, t_max_col]].isna().sum().sum()==0)

In [ ]:
return_df = DOVSAudit.set_i_outg_in_best_ests_df(
    best_ests_df = return_df, 
    groupby_cols = [PN_col], 
    sort_cols    = [t_min_col, t_min_col], 
    i_outg_col   = 'i_outg'
)

In [ ]:
return_df

In [ ]:
best_ests_df

In [ ]:
# def is_PN_suboutg_split_across_DOVS()

In [ ]:
audit_i.best_ests_df_w_keep_info

In [ ]:
ovrlp_audit_j.best_ests_df_w_keep_info

In [ ]:
ovrlp_audit_j.dovs_df_i.iloc[0]['premise_nbs']

In [ ]:
PN_j = '040587963'
be_df_j = audit_i.best_ests_df_w_keep_info[audit_i.best_ests_df_w_keep_info[PN_col]==PN_i]
ovrlp_be_df_j = ovrlp_audit_j.best_ests_df_w_keep_info[ovrlp_audit_j.best_ests_df_w_keep_info[PN_col]==PN_i]

In [ ]:
be_df_i

In [ ]:
ovrlp_be_df_j

In [ ]:
be_df_j[['winner_min', 'winner_max', 'keep']].value_counts().index.tolist()

In [ ]:
be_df_j.groupby(['winner_min', 'winner_max', 'keep']).groups.keys()

In [ ]:
gps       = be_df_j[['winner_min', 'winner_max', 'keep']].value_counts().index.tolist()
ovrlp_gps = ovrlp_be_df_j[['winner_min', 'winner_max', 'keep']].value_counts().index.tolist()

In [ ]:
set(gps).symmetric_difference(ovrlp_gps)

In [ ]:
if set(gps).symmetric_difference(ovrlp_gps)==set():
    # I want to be notified when there are multiple sub-outages
    # Currently coded for single
    assert(len(gps)==len(ovrlp_gps)==1)

In [ ]:
# Look at AMI data and see if power ever lost between two 

In [ ]:
tmp_df = audit_i.ami_df_i[audit_i.ami_df_i['aep_premise_nb']==PN_j].copy()
tmp_df = tmp_df[
    (tmp_df['starttimeperiod_local'] >= be_df_i['winner_min'].min()) & 
    (tmp_df['endtimeperiod_local']   <= be_df_i['winner_max'].max())
].copy()

In [ ]:
(tmp_df['value']!=0).any()

# FOR Peggy slide

In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
# outg_rec_nb = '13551139'
# outg_rec_nb = '13530407'
outg_rec_nb = '13591325'

In [ ]:
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
audit_i = DOVSAudit(
    outg_rec_nb=outg_rec_nb
)
#-------------------------
audit_i.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
audit_i.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
audit_i.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)

In [ ]:
audit_i.build_best_ests_df()
audit_i.best_ests_df

In [ ]:
audit_i.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)

In [ ]:
audit_i.best_ests_df_w_keep_info

In [ ]:
fig_num=0

In [ ]:
best_ests_df = audit_i.best_ests_df.copy()
ami_df_i = audit_i.ami_df_i.copy()
#-----
dovs_outg_t_beg_end = audit_i.dovs_outg_t_beg_end
dovs_outg_t_beg = dovs_outg_t_beg_end[0]
dovs_outg_t_end = dovs_outg_t_beg_end[1]
#-----
outage_nb  = audit_i.outage_nb
n_PNs_dovs = audit_i.n_PNs_dovs
ci_dovs    = audit_i.ci_cmi_dovs[0]
cmi_dovs   = audit_i.ci_cmi_dovs[1]
ci_ami     = best_ests_df['PN'].nunique() 
cmi_ami    = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
threshold=95
#-------------------------
outg_SNs = best_ests_df['PN'].unique().tolist()
ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]
#-------------------------
means_df, best_ests_df_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=True
)
#-------------------------
n_PNs_w_power_srs = DOVSAudit.build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)

In [ ]:
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = means_df, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col='removed_due_to_overlap', 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

fig_num += 1

In [ ]:
expand_time=pd.Timedelta('1 hour')
cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min()
cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max()
#-------------------------
fig, axs = Plot_General.default_subplots(fig_num=fig_num, n_x=1, n_y=2, sharex=True)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))
#-------------------------
palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)
#-------------------------
shared_plot_kwargs = dict(
    x                          = 'starttimeperiod_local', 
    y                          = 'value', 
    hue                        = 'serialnumber', 
    out_t_beg                  = dovs_outg_t_beg, 
    out_t_end                  = dovs_outg_t_end, 
    expand_time                = expand_time, 
    plot_time_beg_end          = [cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
    only_connect_continuous    = False, 
    data_freq                  = None,     
    data_label                 = '', 
    ax_args                    = None, 
    xlabel_args                = None, 
    ylabel_args                = None, 
    df_mean                    = None, 
    df_mean_col                = None, 
    mean_args                  = None, 
    draw_outage_limits         = True, 
    draw_outage_limits_kwargs  = dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
    include_outage_limits_text = dict(
        out_t_beg_text  = 'DOVS Beg.', 
        out_t_beg_ypos  = (0.12, 'ax_coord'), 
        out_t_beg_va    = 'bottom', 
        out_t_beg_ha    = 'center', 
        out_t_beg_color = 'red', 
        #-----
        out_t_end_text  = 'DOVS End', 
        out_t_end_ypos  = (0.12, 'ax_coord'), 
        out_t_end_va    = 'bottom', 
        out_t_end_ha    = 'center', 
        out_t_end_color = 'green', 
    ), 
    draw_without_hue_also      = False, 
    seg_line_freq              = None, 
    palette                    = palette
)
#-----
i_subplot=0
fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=axs[i_subplot], 
    data=ami_df_i, 
    title_args=dict(label=f"#SNs = {ami_df_i['serialnumber'].nunique()}", fontdict=dict(fontsize=24)), 
    **shared_plot_kwargs
)
#-----
add_all_best_ests_to_axis(
    axs[i_subplot], 
    means_df, 
    line_kwargs_by_est_key=dict(
        conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
        zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
    ), 
    keys_to_include=['winner', 'conservative', 'zero_times']
)
#-----
axs[i_subplot].legend().set_visible(False)
Plot_General.set_general_plotting_args(
    ax=axs[i_subplot], 
    tick_args =[
        dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
        dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
    ], 
    xlabel_args=dict(xlabel='15T Interval Start Time', fontsize=16), 
    ylabel_args=dict(ylabel='Voltage (V)', fontsize=16)
)
#-------------------------
i_subplot=1
fig,axs[i_subplot] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[i_subplot]), 
    threshold_color='magenta'
)
#-----
Plot_General.set_general_plotting_args(
    ax=axs[i_subplot], 
    tick_args =[
        dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
        dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
    ], 
    xlabel_args=dict(xlabel='15T Interval Start Time', fontsize=16), 
    ylabel_args=dict(ylabel='%PNs with Power', fontsize=16)
)
#-------------------------
fig_num += 1

#-------------------------
Plot_General.save_fig(
    fig=fig, 
    save_dir=r'C:\Users\s346557\Documents\Presentations\ForPeggy_Nov2023', 
    save_name='Figs_ab_2.png'
)

In [ ]:
expand_time=pd.Timedelta('1 hour')
cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min()
cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max()
#-------------------------
fig, axs = Plot_General.default_subplots(fig_num=fig_num, n_x=1, n_y=1, sharex=True)
axs=[axs]
Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))
#-------------------------
palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)
#-------------------------
shared_plot_kwargs = dict(
    x                          = 'starttimeperiod_local', 
    y                          = 'value', 
    hue                        = 'serialnumber', 
    out_t_beg                  = dovs_outg_t_beg, 
    out_t_end                  = dovs_outg_t_end, 
    expand_time                = expand_time, 
    plot_time_beg_end          = [cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
    only_connect_continuous    = False, 
    data_freq                  = None,     
    data_label                 = '', 
    ax_args                    = None, 
    xlabel_args                = None, 
    ylabel_args                = None, 
    df_mean                    = None, 
    df_mean_col                = None, 
    mean_args                  = None, 
    draw_outage_limits         = True, 
    draw_outage_limits_kwargs  = dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
    include_outage_limits_text = dict(
        out_t_beg_text  = 'DOVS Beg.', 
        out_t_beg_ypos  = (0.12, 'ax_coord'), 
        out_t_beg_va    = 'bottom', 
        out_t_beg_ha    = 'center', 
        out_t_beg_color = 'red', 
        #-----
        out_t_end_text  = 'DOVS End', 
        out_t_end_ypos  = (0.12, 'ax_coord'), 
        out_t_end_va    = 'bottom', 
        out_t_end_ha    = 'center', 
        out_t_end_color = 'green', 
    ), 
    draw_without_hue_also      = False, 
    seg_line_freq              = None, 
    palette                    = palette
)
#-----
i_subplot=0
fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=axs[i_subplot], 
    data=ami_df_i, 
    title_args=dict(label=f"#SNs = {ami_df_i['serialnumber'].nunique()}", fontdict=dict(fontsize=24)), 
    **shared_plot_kwargs
)
#-----
add_all_best_ests_to_axis(
    axs[i_subplot], 
    means_df, 
    line_kwargs_by_est_key=dict(
        conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
        zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
    ), 
    keys_to_include=['winner', 'conservative', 'zero_times']
)
#-----
axs[i_subplot].legend().set_visible(False)
Plot_General.set_general_plotting_args(
    ax=axs[i_subplot], 
    tick_args =[
        dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
        dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
    ], 
    xlabel_args=dict(xlabel='15T Interval Start Time', fontsize=16), 
    ylabel_args=dict(ylabel='Voltage (V)', fontsize=16)
)
#-------------------------
fig_num += 1

#-------------------------
Plot_General.save_fig(
    fig=fig, 
    save_dir=r'C:\Users\s346557\Documents\Presentations\ForPeggy_Nov2023', 
    save_name='Fig_a_2.png'
)

In [ ]:
means_df

In [ ]:
dovs_outg_t_beg

In [ ]:
dovs_outg_t_end

In [ ]:
pd.to_datetime('2023-09-11 14:08:00')-(pd.Timedelta('13T')+pd.Timedelta('27s'))

# DEV

In [ ]:
# outg_rec_nb = '13294714'
outg_rec_nb = '13464904'

In [ ]:
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
audit_i = DOVSAudit(
    outg_rec_nb=outg_rec_nb
)
#-------------------------
audit_i.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
audit_i.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
audit_i.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)

In [ ]:
audit_i.build_best_ests_df()
audit_i.best_ests_df

In [ ]:
audit_i.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)

In [ ]:
audit_i.resolve_overlapping_audits()

In [ ]:
best_ests_df = audit_i.best_ests_df.copy()

In [ ]:
best_ests_df

In [ ]:
likeness_thresh = pd.Timedelta('1 minutes')
SN_col = 'SN'
PN_col = 'PN'
i_outg_col = 'i_outg'     

In [ ]:
minmax_cols = [x for x in best_ests_df.columns.tolist() if x not in [SN_col, PN_col, i_outg_col]]
min_cols = [x for x in best_ests_df.columns.tolist() if x.endswith('_min')]
max_cols = [x for x in best_ests_df.columns.tolist() if x.endswith('_max')]

In [ ]:
minmax_cols

In [ ]:
%run ./check_DOVS_METHODS.ipynb
fig_num=0

In [ ]:
best_ests_df = audit_i.best_ests_df.copy()
ami_df_i = audit_i.ami_df_i.copy()
#-----
dovs_outg_t_beg_end = audit_i.dovs_outg_t_beg_end
dovs_outg_t_beg = dovs_outg_t_beg_end[0]
dovs_outg_t_end = dovs_outg_t_beg_end[1]
#-----
outage_nb  = audit_i.outage_nb
n_PNs_dovs = audit_i.n_PNs_dovs
ci_dovs    = audit_i.ci_cmi_dovs[0]
cmi_dovs   = audit_i.ci_cmi_dovs[1]
ci_ami     = best_ests_df['PN'].nunique() 
cmi_ami    = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
threshold=95
#-------------------------
outg_SNs = best_ests_df['PN'].unique().tolist()
ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]
#-------------------------
means_df, best_ests_df_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=True
)
#-------------------------
n_PNs_w_power_srs = DOVSAudit.build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)

In [ ]:
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = means_df, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

fig_num += 1

# Sample plot: Overlapping sub-outages

In [ ]:
search_time_half_window=pd.Timedelta('24 hours')
#-------------------------
audit_1 = DOVSAudit(
    outg_rec_nb='13686882'
)
#-------------------------
audit_1.load_dovs(
    dovs_df           = None, 
    dovs_df_info_dict = None
)
#-------------------------
audit_1.build_mp_df(
    drop_mp_dups_fuzziness=pd.Timedelta('1 hour')
)
#-------------------------
audit_1.run_ami_and_ede_daq(
    search_time_half_window=search_time_half_window
)
#-------------------------
audit_1.build_best_ests_df()
#-------------------------


In [ ]:
fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage_around_outage(
    fig=fig,
    ax=ax,
    data=audit_1.ami_df_i,
    x='starttimeperiod_local',
    y='value',
    hue='serialnumber',
    out_t_beg=audit_1.dovs_outg_t_beg_end[0],
    out_t_end=audit_1.dovs_outg_t_beg_end[1],
    expand_time=pd.Timedelta('1h'),
)
ax.legend().set_visible(False)

In [ ]:
audit_1.identify_overlaps(overlaps_dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage)

In [ ]:
audit_1.resolve_overlapping_audits()

In [ ]:
audit_i=copy.deepcopy(audit_1)

In [ ]:
best_ests_df = audit_i.best_ests_df.copy()
ami_df_i = audit_i.ami_df_i.copy()
#-----
dovs_outg_t_beg_end = audit_i.dovs_outg_t_beg_end
dovs_outg_t_beg = dovs_outg_t_beg_end[0]
dovs_outg_t_end = dovs_outg_t_beg_end[1]
#-----
outage_nb  = audit_i.outage_nb
n_PNs_dovs = audit_i.n_PNs_dovs
ci_dovs    = audit_i.ci_cmi_dovs[0]
cmi_dovs   = audit_i.ci_cmi_dovs[1]
ci_ami     = best_ests_df['PN'].nunique() 
cmi_ami    = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60
threshold=95
#-------------------------
outg_SNs = best_ests_df['PN'].unique().tolist()
ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]
#-------------------------
means_df, best_ests_df_w_db_lbl = DOVSAudit.get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=True
)
#-------------------------
n_PNs_w_power_srs = DOVSAudit.build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)

In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
fig_num=0
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = means_df, 
    outg_rec_nb          = audit_i.outg_rec_nb, 
    outage_nb            = audit_i.outage_nb, 
    n_PNs_dovs           = audit_i.n_PNs_dovs, 
    ci_dovs              = audit_i.ci_cmi_dovs[0], 
    cmi_dovs             = audit_i.ci_cmi_dovs[0], 
    ci_ami               = audit_i.ci, 
    cmi_ami              = audit_i.cmi, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

for ax_i in axs:
    ax_i.xaxis.set_tick_params(labelbottom=True)

# Plot_General.save_fig(
#     fig=fig, 
#     save_dir=r'C:\Users\s346557\Documents\Presentations\DOVS_check\_DemoFigs\Overlap_suboutgs', 
#     save_name='Overlap_suboutgs_CORRECT.png', 
#     bbox_inches='tight'
# )

fig_num += 1

In [ ]:
fig_num=0
best_ests_df_i = best_ests_df[best_ests_df['PN']=='023368334']
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i[ami_df_i['aep_premise_nb']=='023368334'], 
    ami_df_i_out         = ami_df_i_out[ami_df_i_out['aep_premise_nb']=='023368334'], 
    ami_df_i_not_out     = ami_df_i_not_out[ami_df_i_not_out['aep_premise_nb']=='023368334'], 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = best_ests_df_i, 
    outg_rec_nb          = audit_i.outg_rec_nb, 
    outage_nb            = audit_i.outage_nb, 
    n_PNs_dovs           = audit_i.n_PNs_dovs, 
    ci_dovs              = audit_i.ci_cmi_dovs[0], 
    cmi_dovs             = audit_i.ci_cmi_dovs[0], 
    ci_ami               = audit_i.ci, 
    cmi_ami              = audit_i.cmi, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)
axs[1].set_title(f"CMI = {np.round((best_ests_df_i['winner_max']-best_ests_df_i['winner_min']).sum().total_seconds()/60, 2)}", fontdict=dict(fontsize=24))

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

for ax_i in axs:
    ax_i.xaxis.set_tick_params(labelbottom=True)

# Plot_General.save_fig(
#     fig=fig, 
#     save_dir=r'C:\Users\s346557\Documents\Presentations\DOVS_check\_DemoFigs\Overlap_suboutgs', 
#     save_name='Overlap_suboutgs_CORRECT_i.png', 
#     bbox_inches='tight'
# )

fig_num += 1

In [ ]:
best_ests_df

In [ ]:
best_ests_df_i

In [ ]:
fig_num=0 
best_ests_df_i2 = best_ests_df[best_ests_df['PN']=='020115172']
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i[ami_df_i['aep_premise_nb']=='020115172'], 
    ami_df_i_out         = ami_df_i_out[ami_df_i_out['aep_premise_nb']=='020115172'], 
    ami_df_i_not_out     = ami_df_i_not_out[ami_df_i_not_out['aep_premise_nb']=='020115172'], 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df['conservative_max'].max(), 
    means_df             = best_ests_df_i2, 
    outg_rec_nb          = audit_i.outg_rec_nb, 
    outage_nb            = audit_i.outage_nb, 
    n_PNs_dovs           = audit_i.n_PNs_dovs, 
    ci_dovs              = audit_i.ci_cmi_dovs[0], 
    cmi_dovs             = audit_i.ci_cmi_dovs[0], 
    ci_ami               = audit_i.ci, 
    cmi_ami              = audit_i.cmi, 
    name                 = 'AMI', 
    results_2_dict       = None, 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col=None, 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True, sharex=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)
axs[1].set_title(f"CMI = {np.round((best_ests_df_i2['winner_max']-best_ests_df_i2['winner_min']).sum().total_seconds()/60, 2)}", fontdict=dict(fontsize=24))

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

for ax_i in axs:
    ax_i.xaxis.set_tick_params(labelbottom=True)

# Plot_General.save_fig(
#     fig=fig, 
#     save_dir=r'C:\Users\s346557\Documents\Presentations\DOVS_check\_DemoFigs\Overlap_suboutgs', 
#     save_name='Overlap_suboutgs_CORRECT_i2.png', 
#     bbox_inches='tight'
# )

fig_num += 1

In [ ]:
best_ests_df_i2